In [14]:
from pexels_apis import PexelsAPI
import os
from dotenv import load_dotenv
import json
from react_agent.utils import extract_video_data
load_dotenv()

True

In [15]:
pexels = PexelsAPI(os.environ.get('PEXELS_API_KEY'))

In [22]:
search_params = {
    "query": "a person surrounded with water",
     
    "orientation": "portrait",
    "size": "medium",           
    "page": 1,                 
    "per_page": 30
}

videos = pexels.search_videos(search_params)
# json_output = json.dumps(videos, indent=4)
# print(json_output)
clean_data = extract_video_data(videos)
from pprint import pprint
pprint(clean_data)


[{'author': 'ROMAN ODINTSOV',
  'author_url': 'https://www.pexels.com/@roman-odintsov',
  'duration': 11,
  'height': 4096,
  'id': 8195680,
  'preview_image': 'https://images.pexels.com/videos/8195680/blooming-bright-color-dawn-8195680.jpeg?auto=compress&cs=tinysrgb&fit=crop&h=1200&w=630',
  'video_files': [{'file_type': 'video/mp4',
                   'fps': 29.969999313354492,
                   'height': 4096,
                   'link': 'https://videos.pexels.com/video-files/8195680/8195680-uhd_2160_4096_30fps.mp4',
                   'quality': 'uhd',
                   'size': 31569170,
                   'width': 2160},
                  {'file_type': 'video/mp4',
                   'fps': 29.969999313354492,
                   'height': 2048,
                   'link': 'https://videos.pexels.com/video-files/8195680/8195680-hd_1080_2048_30fps.mp4',
                   'quality': 'hd',
                   'size': 7655049,
                   'width': 1080},
                  {'file_

In [31]:
import re

def extract_video_name(url: str) -> str:
    """
    Extracts the descriptive name from a Pexels video URL.
    Example: "https://www.pexels.com/video/a-man-made-pond-surrounded-by-rocks-and-plants-8195680/"
             → "A Man Made Pond Surrounded By Rocks And Plants"
    """
    match = re.search(r'/video/([^/]+)-\d+/?$', url)
    if match:
        return match.group(1).replace('-', ' ').title()
    return "Untitled"

# Assume `clean_data` is your list of video dictionaries
video_dict = {
    str(video['id']): extract_video_name(video['video_url'])
    for video in clean_data
}

from pprint import pprint
pprint(video_dict)


{'17076996': 'The Sun Setting Over The Water On A Boat',
 '17949249': 'Sao Conrado Rj',
 '17982228': 'Woman Walking On The Sand Beach',
 '20257643': 'Two People Walking On The Beach At Sunset',
 '27989388': 'A Woman In A Gold Dress Posing On The Rocks',
 '28241358': 'A Woman Standing By The Water In A Brown Cardigan',
 '28557546': 'A Man Standing On The Beach Watching The Sunset',
 '30321490': 'Serene Beach Sunset With Calm Waves',
 '31879279': 'Scuba Divers Exploring Underwater Canyon',
 '31884015': 'Spring Walk Over Bridge In Urban Park Setting',
 '31892609': 'Scenic Lighthouse At Dusk With Sky Trail',
 '31930531': 'Serene Evening Swim In Coastal Waters',
 '4434370': 'A Person Walking On The Seashore',
 '4759582': 'Woman Walking On White Sand Of Sea Water',
 '4762777': 'Woman Walking Under The Water Falls',
 '4936970': 'Person Pose And Laying On A Tree Trunk In The Forest',
 '5010624': 'People Having A Vacation',
 '5057325': 'Motor Boats With Tourists At Venice City',
 '5201403': 'Wo

In [5]:
# pexels.get_video(5520073)

In [10]:
# The JSON object representing the script
script_data = {
    "title": "Why You Feel Like a Fraud—Even When You're Winning",
    "length": "90-120 seconds",
    "background_music": {
        "music": "subtle motivational"
    },
    "sections": [
        {
            "section": "HOOK",
            "text": "You crushed the presentation. Got the promotion. Nailed the project. But deep down, you're waiting for someone to tap you on the shoulder and say, 'We made a mistake.'",
            "visual": {
                "scene": "woman sitting at desk staring at a 'Employee of the Month' certificate",
                "camera_angle": "close-up of conflicted facial expression",
                "transition": "cut to",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": "1 second",
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "CONCEPT",
            "text": "That voice whispering, 'You don’t belong here'? It’s not imposter syndrome—it’s your brain replaying old tapes. Tapes from a time when your worth was measured by what you did, not who you are.",
            "visual": {
                "scene": "man walking alone in a park, looking down at his feet",
                "camera_angle": "side profile in moody lighting",
                "transition": "fade to black",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "REAL-WORLD EXAMPLE",
            "text": "You’re at a team meeting. Your idea gets praised. But instead of feeling proud, you think, 'They’re just being nice.' Later, you overhear a colleague say, 'She’s so confident.' You laugh bitterly. If only they knew. You replay every mistake, every hesitation, every moment you didn’t feel 'enough.' You work late—again—to prove you deserve the seat you’re already sitting in. But the more you achieve, the louder the doubt gets. It’s a loop: perform, doubt, overcompensate, repeat.",
            "visual": {
                "scene": "woman in a boardroom, colleagues clapping, but she looks uneasy",
                "camera_angle": "over-the-shoulder shot focusing on her discomfort",
                "transition": "cut to",
                "sound": {
                    "sound_effects": "faint office chatter",
                    "silence_duration": None,
                    "sound_effect_timing": "as the clapping starts"
                }
            }
        },
        {
            "section": "PSYCHOLOGICAL INSIGHT",
            "text": "This isn’t about your skills—it’s about internalized shame. You were conditioned to believe your value is tied to your output. So now, even when you succeed, your brain screams, 'Fraud!' because it doesn’t recognize worth without struggle.",
            "visual": {
                "scene": "person staring at their reflection in a window, looking lost",
                "camera_angle": "close-up of reflection with blurred background",
                "transition": "dissolve",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "ACTIONABLE TIP",
            "text": "Start a 'proof of worth' journal. Every day, write one thing you did—not because you had to, but because you chose to. Not for praise, but for joy. Slowly, you’ll rewire your brain to see your value beyond productivity.",
            "visual": {
                "scene": "person writing in a notebook at a cozy café table",
                "camera_angle": "overhead shot of hands writing",
                "transition": "fade to black",
                "sound": {
                    "sound_effects": "pen scratching paper",
                    "silence_duration": None,
                    "sound_effect_timing": "as the writing begins"
                }
            }
        },
        {
            "section": "CTA",
            "text": "Follow for more psychology that helps you silence the fraud narrative—and start owning your worth.",
            "visual": {
                "scene": "person walking confidently down a city street, sunlight breaking through clouds",
                "camera_angle": "wide shot with uplifting lighting",
                "transition": "cut to black",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        }
    ]
}

In [11]:
from pathlib import Path
import requests
import json
from typing import Dict
from react_agent.structures import VideoScript
from react_agent.utils import extract_video_data  # Adjust path as needed
import re

def sanitize_filename(name: str) -> str:
    """
    Convert a string to a safe filename:
    - Replace spaces with underscores
    - Remove non-alphanumeric characters (except underscores and dashes)
    - Collapse multiple underscores
    """
    name = name.replace(" ", "_")
    name = re.sub(r"[^\w\-]", "", name)  # Keep alphanumerics, underscores, dashes
    name = re.sub(r"_+", "_", name)  # Collapse multiple underscores
    return name.strip("_")


async def get_videos_from_script(script_data: Dict) -> Dict:
    """Search Pexels for videos matching each visual scene and download them."""

    try:
        script_obj = VideoScript(**script_data)
    except Exception as e:
        return {"status": "error", "message": f"Invalid script data: {e}"}

    # Create a sanitized directory for the video set
    safe_title = sanitize_filename(script_obj.title)
    video_dir = Path(f"my_test_files/videos/{safe_title}/")

    video_dir.mkdir(parents=True, exist_ok=True)

    results = []

    for section in script_obj.sections:
        search_query = section.visual.scene

        search_params = {
            "query": search_query,
            "orientation": "portrait",
            "size": "medium",
            "page": 1,
            "per_page": 5,
        }

        pexels_response = pexels.search_videos(search_params)  # Use actual object

        if pexels_response.get("status_code") != 200:
            print(f"Failed to find video for: {search_query}")
            continue

        videos_data = extract_video_data(pexels_response)
        if not videos_data:
            print(f"No videos returned for: {search_query}")
            continue

        # print(f'videos data: {videos_data}\n')
        best_video = videos_data[0]
        target_aspect = 9 / 16

        # Filter to SD quality portrait-oriented videos
        sd_videos = [
            v for v in best_video["video_files"]
            if v["quality"] == "sd" and v["width"] / v["height"] <= target_aspect
        ]
        if not sd_videos:
            sd_videos = [v for v in best_video["video_files"] if v["quality"] == "sd"]

        if not sd_videos:
            print(f"No suitable SD video found for: {search_query}")
            continue

        download_video = min(
            sd_videos, key=lambda v: abs((v["width"] / v["height"]) - target_aspect)
        )
        print(f'download video: {download_video}\n')
        # Generate sanitized filename
        filename = sanitize_filename(f"{safe_title}_{section.section}") + ".mp4"
        video_path = video_dir /  filename

        try:
            response = requests.get(download_video["link"], stream=True)
            response.raise_for_status()

            with open(video_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            video_metadata = {
                "script_section": section.section,
                "pexels_id": best_video["id"],
                "author": best_video["author"],
                "author_url": best_video["author_url"],
                "video_url": best_video["video_url"],
                "download_url": download_video["link"],
                "file_path": str(video_path),
                "dimensions": f"{download_video['width']}x{download_video['height']}",
                "duration": best_video["duration"],
                "search_query": search_query,
                "attribution": f"Video by {best_video['author']} from Pexels",
            }

            results.append(video_metadata)

        except Exception as e:
            print(f"Failed to download video for {search_query}: {str(e)}")
            continue

    metadata_path = video_dir / "video_metadata.json"
    with open(metadata_path, "w") as f:
        json.dump(results, f, indent=4)

    return {
        "status": "success",
        "downloaded_videos": len(results),
        "total_sections": len(script_obj.sections),
        "video_directory": str(video_dir),
        "metadata_file": str(metadata_path),
        "videos": results,
    }


In [12]:
k = await get_videos_from_script(script_data)

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/8547568/8547568-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 567407}

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/6132796/6132796-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 961748}

download video: {'file_type': 'video/mp4', 'width': 360, 'height': 640, 'link': 'https://videos.pexels.com/video-files/8135918/8135918-sd_360_640_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 759532}

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/7207576/7207576-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 899090}

download video: {'file_type': 'video/mp4', 'width': 240, 'height': 408, 'link': 'https://videos.pexels.com/video-files/4460555/4460555-sd_240_408_25fps.mp4', 'quality': 'sd', 'fps'

In [13]:
k

{'status': 'success',
 'downloaded_videos': 6,
 'total_sections': 6,
 'video_directory': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning',
 'metadata_file': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/video_metadata.json',
 'videos': [{'script_section': 'HOOK',
   'pexels_id': 8547568,
   'author': 'Photo By: Kaboompics.com',
   'author_url': 'https://www.pexels.com/@karolina-grabowska',
   'video_url': 'https://www.pexels.com/video/a-tired-woman-looking-at-her-laptop-8547568/',
   'download_url': 'https://videos.pexels.com/video-files/8547568/8547568-sd_226_426_25fps.mp4',
   'file_path': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning_HOOK.mp4',
   'dimensions': '226x426',
   'duration': 19,
   'search_query': "woman sitting at desk staring at a 'Employee of the Month' certificate",
   'attribution': 'Video by Photo By: Kaboompics.com from Pexels'},
  {'script_sec

In [26]:
k['metadata_file']

'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/video_metadata.json'

# ctc aligner

In [1]:
# import torch
# from ctc_forced_aligner import (
#     load_audio,
#     load_alignment_model,
#     generate_emissions,
#     preprocess_text,
#     get_alignments,
#     get_spans,
#     postprocess_results,
# )

# audio_path = "my_test_files/audios/one.wav"
# text_path = "my_test_files/audios/one.txt"
# language = "iso" # ISO-639-3 Language code
# device = "cuda" if torch.cuda.is_available() else "cpu"
# batch_size = 16


# alignment_model, alignment_tokenizer = load_alignment_model(
#     device,
#     dtype=torch.float16 if device == "cuda" else torch.float32,
# )

# audio_waveform = load_audio(audio_path, alignment_model.dtype, alignment_model.device)


# with open(text_path, "r") as f:
#     lines = f.readlines()
# text = "".join(line for line in lines).replace("\n", " ").strip()

# emissions, stride = generate_emissions(
#     alignment_model, audio_waveform, batch_size=batch_size
# )

# tokens_starred, text_starred = preprocess_text(
#     text,
#     romanize=True,
#     language=language,
# )

# segments, scores, blank_token = get_alignments(
#     emissions,
#     tokens_starred,
#     alignment_tokenizer,
# )

# spans = get_spans(tokens_starred, segments, blank_token)

# word_timestamps = postprocess_results(text_starred, spans, stride, scores)

# kokoro-onnx

In [3]:

import soundfile as sf
from misaki import en, espeak

from kokoro_onnx import Kokoro

# Misaki G2P with espeak-ng fallback
fallback = espeak.EspeakFallback(british=False)
g2p = en.G2P(trf=False, british=False, fallback=fallback)

# Kokoro
kokoro = Kokoro("/home/aditya-ladawa/Aditya/z_projects/short_creation/kokoro_files/kokoro.onnx", "/home/aditya-ladawa/Aditya/z_projects/short_creation/kokoro_files/voices-v1.0.bin")

# Phonemize
text = "AT the shoreline...... I found a way to swim !! In sands. Wait wait, .... Am I .....smart or what?"
phonemes, _ = g2p(text)

# Create
samples, sample_rate = kokoro.create(phonemes, "af_heart", is_phonemes=True)

# Save
sf.write("audio.wav", samples, sample_rate)
print("Created audio.wav")

Created audio.wav


In [3]:
# ! wget https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/voices-v1.0.bin

## FFMPEG

In [ ]:
import ffmpeg
stream = ffmpeg.input('/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Hidden_Trap_of_Survivorship_BiasAnd_How_Its_Lying_to_You/visuals/ACTIONABLE_TIP_6940831.mp4')
stream = ffmpeg.filter_multi_output

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

(None, None)

In [6]:
help(ffmpeg.filter_multi_output)

Help on function filter_multi_output in module ffmpeg._filters:

filter_multi_output(stream_spec, filter_name, *args, **kwargs)
    Apply custom filter with one or more outputs.

    This is the same as ``filter`` except that the filter can produce more than one output.

    To reference an output stream, use either the ``.stream`` operator or bracket shorthand:

    Example:

        ```
        split = ffmpeg.input('in.mp4').filter_multi_output('split')
        split0 = split.stream(0)
        split1 = split[1]
        ffmpeg.concat(split0, split1).output('out.mp4').run()
        ```



In [ ]:
import ffmpeg
import random
import os

def get_duration(filename):
    """Gets the duration of a media file."""
    try:
        probe = ffmpeg.probe(filename)
        duration = float(probe['format']['duration'])
        return duration
    except ffmpeg.Error as e:
        print(f"FFmpeg error: {e.stderr.decode('utf8')}")
        return None

def process_audio_video_pair(audio_file, video_files, output_file):
    """Processes one audio file and its associated video files."""
    audio_duration = get_duration(audio_file)
    if audio_duration is None:
        return

    print(f"Audio duration: {audio_duration} seconds")

    segments_to_concat = []
    current_video_index = 0
    total_video_duration = 0

    while total_video_duration < audio_duration:
        if not video_files:
            print("No video files available to reach audio duration.")
            break

        current_video = video_files[current_video_index % len(video_files)]
        video_duration = get_duration(current_video)
        if video_duration is None:
            video_files.pop(current_video_index % len(video_files)) # Remove problematic file
            continue

        # Determine a random start time (within the video's duration)
        max_start = max(0, video_duration - 1) # Ensure we can get at least a short segment
        start_time = random.uniform(0, max_start)

        # Determine a random segment duration (but don't exceed remaining audio duration or video length)
        remaining_duration = audio_duration - total_video_duration
        max_segment_duration = min(video_duration - start_time, remaining_duration, 5) # Example max segment of 5 seconds
        if max_segment_duration <= 0:
            current_video_index += 1
            continue

        segment_duration = random.uniform(0.5, max_segment_duration) # Example min 0.5 sec segment

        input_node = ffmpeg.input(current_video, ss=start_time, t=segment_duration)
        segments_to_concat.append(input_node)
        total_video_duration += segment_duration
        current_video_index += 1

        print(f"  - Added segment from {os.path.basename(current_video)} at {start_time:.2f} for {segment_duration:.2f} seconds. Total: {total_video_duration:.2f}")

    if segments_to_concat:
        # Concatenate the video segments
        concatenated_video = ffmpeg.concat(*segments_to_concat, v=1, a=0)

        # Add the audio
        audio_input = ffmpeg.input(audio_file)
        output_node = ffmpeg.output(concatenated_video, audio_input, output_file, acodec='copy') # Or specify audio codec
        try:
            ffmpeg.run(output_node)
            print(f"Successfully created: {output_file}")
        except ffmpeg.Error as e:
            print(f"FFmpeg error during output: {e.stderr.decode('utf8')}")
    else:
        print(f"Could not create video for {os.path.basename(audio_file)}.")

# Example usage (assuming you have lists of audio and corresponding video files)
audio_files = ['audio1.wav', 'audio2.wav']
video_sets = [['video1_a.mp4', 'video1_b.mp4', 'video1_c.mp4'],
              ['video2_a.mov', 'video2_b.avi']]
output_dir = 'output_videos'
os.makedirs(output_dir, exist_ok=True)

for i, audio in enumerate(audio_files):
    output_filename = os.path.join(output_dir, f'output_{os.path.splitext(os.path.basename(audio))[0]}.mp4')
    process_audio_video_pair(audio, video_sets[i], output_filename)

In [ ]:
get_duration('/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Hidden_Trap_of_Survivorship_BiasAnd_How_Its_Lying_to_You/audio/HOOK.wav')

9.173333

In [11]:
os.listdir('/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Hidden_Trap_of_Survivorship_BiasAnd_How_Its_Lying_to_You/visuals')

['REAL-WORLD_EXAMPLE_9064795.mp4',
 'REAL-WORLD_EXAMPLE_7172723.mp4',
 'video_metadata.json',
 'PSYCHOLOGICAL_INSIGHT_6003998.mp4',
 'CTA_5636171.mp4',
 'PSYCHOLOGICAL_INSIGHT_6954284.mp4',
 'PSYCHOLOGICAL_INSIGHT_7392450.mp4',
 'ACTIONABLE_TIP_6940831.mp4',
 'HOOK_8165696.mp4',
 'CTA_7241762.mp4',
 'CTA_7578123.mp4',
 'HOOK_7967298.mp4',
 'REAL-WORLD_EXAMPLE_6913257.mp4',
 'ACTIONABLE_TIP_7576277.mp4',
 'CONCEPT_8035633.mp4',
 'HOOK_6799753.mp4',
 'ACTIONABLE_TIP_7062989.mp4',
 'CONCEPT_8035616.mp4',
 'CONCEPT_6964002.mp4']

In [1]:
import ffmpeg
import random
import os
import glob # For easier file matching

# --- Configuration ---
# Adjust these paths and settings as needed
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/"
OUTPUT_DIR_BASE = "output_reels_fades_only" # Directory to save the final reels

MIN_SEGMENT_DURATION = 2.0  # Minimum duration for a video segment in seconds
MAX_SEGMENT_DURATION = 5.0  # Maximum duration for a video segment in seconds
TRANSITION_DURATION = 0.5   # Duration of transitions between segments in seconds

# Instagram Reel settings
REEL_WIDTH = 720           # Output video width (1080 for higher quality)
REEL_HEIGHT = 1280          # Output video height (1920 for higher quality)
OUTPUT_FPS = 30             # Output video frames per second

# Encoding settings
VIDEO_CODEC = 'libx264'     # H.264 is widely compatible
AUDIO_CODEC = 'aac'         # AAC is standard for audio
AUDIO_BITRATE = '128k'      # Good audio quality for speech
CRF = 23                    # Constant Rate Factor for x264 (18-28 is a good range, lower is better quality but larger file)
PRESET_X264 = 'medium'      # Encoding speed vs. compression (e.g., 'ultrafast', 'medium', 'slow')

# Only use 'fade' transition
XFADE_TRANSITIONS = ['fade']

# --- Helper Functions ---

def get_duration(filename):
    """Gets the duration of a media file."""
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except ffmpeg.Error as e:
        print(f"FFmpeg error probing duration for {filename}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
        return None
    except Exception as e:
        print(f"Error probing duration for {filename}: {str(e)}")
        return None

def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    """
    Creates a reel for a single audio file by processing and combining video segments.
    """
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None:
        print(f"Could not get duration for audio: {audio_file_path}. Skipping.")
        return
    if not associated_video_files:
        print(f"No video files found for audio: {audio_file_path}. Skipping.")
        return

    print(f"\nProcessing audio: {os.path.basename(audio_file_path)}, Target Duration: {audio_duration:.2f}s")

    processed_segments_info = []  # Stores {'stream': ffmpeg_stream, 'duration': float_seconds}
    current_total_video_duration = 0
    video_file_idx = 0
    
    local_video_files = list(associated_video_files) 
    random.shuffle(local_video_files)

    # --- Inner helper to process each segment ---
    def apply_segment_effects(video_node, segment_actual_duration, video_path_for_log=""):
        """
        Applies essential scaling, cropping, FPS, and PTS reset. No visual effects.
        """
        
        # 1. Scale to cover and crop to 9:16 using individual filter calls
        processed_node = video_node.filter('scale',
                                           w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                                           h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                                           eval='frame') 
        processed_node = processed_node.filter('crop',
                                               w=REEL_WIDTH,
                                               h=REEL_HEIGHT,
                                               x=f'(iw-{REEL_WIDTH})/2', 
                                               y=f'(ih-{REEL_HEIGHT})/2')
        
        # 2. Ensure consistent FPS for the segment and reset timestamps for concatenation
        processed_node = processed_node.filter('fps', fps=OUTPUT_FPS, round='near')
        processed_node = processed_node.filter('setpts', 'PTS-STARTPTS')
        return processed_node
    # --- End of inner helper ---

    while current_total_video_duration < audio_duration:
        if not local_video_files:
            print("Warning: Ran out of video files while processing.")
            break

        current_video_path = local_video_files[video_file_idx % len(local_video_files)]
        clip_native_duration = get_duration(current_video_path)

        if clip_native_duration is None or clip_native_duration < 0.5:
            video_file_idx += 1
            if video_file_idx >= len(local_video_files) * 2:
                print("Warning: Many problematic video files. May result in short output.")
                break
            continue

        remaining_audio_time = audio_duration - current_total_video_duration
        
        possible_segment_duration = random.uniform(MIN_SEGMENT_DURATION, MAX_SEGMENT_DURATION)
        segment_duration_capped_by_clip = clip_native_duration
        
        actual_segment_duration = min(possible_segment_duration, remaining_audio_time, segment_duration_capped_by_clip)

        if actual_segment_duration < MIN_SEGMENT_DURATION and remaining_audio_time > MIN_SEGMENT_DURATION:
            actual_segment_duration = min(MIN_SEGMENT_DURATION, remaining_audio_time, segment_duration_capped_by_clip)
        
        if actual_segment_duration < 0.2: 
            if current_total_video_duration > 0:
                 break
            video_file_idx += 1
            continue
        
        max_start_time = clip_native_duration - actual_segment_duration
        start_time = random.uniform(0, max(0, max_start_time))

        print(f"  Segment from {os.path.basename(current_video_path)}: ss={start_time:.2f}s, t={actual_segment_duration:.2f}s")

        try:
            # Inputting with ss and t creates a playing segment
            input_node = ffmpeg.input(current_video_path, ss=start_time, t=actual_segment_duration)
            processed_video_stream = apply_segment_effects(input_node.video, actual_segment_duration, current_video_path)
            
            processed_segments_info.append({'stream': processed_video_stream, 'duration': actual_segment_duration})
            current_total_video_duration += actual_segment_duration
            video_file_idx += 1

        except ffmpeg.Error as e:
            print(f"    FFmpeg error processing segment from {current_video_path}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
            video_file_idx += 1
            continue 
        except Exception as e:
            print(f"    Unexpected error processing segment: {str(e)}")
            video_file_idx += 1
            continue

        if current_total_video_duration >= audio_duration:
            overshoot = current_total_video_duration - audio_duration
            if overshoot > 0 and processed_segments_info:
                last_segment_info = processed_segments_info[-1]
                # Adjust duration in info; actual stream trimming relies on -shortest or final output duration
                last_segment_info['duration'] = max(0.1, last_segment_info['duration'] - overshoot) # ensure duration is not zero or negative
                current_total_video_duration = sum(s['duration'] for s in processed_segments_info)

                print(f"    Adjusted segment durations. New total: {current_total_video_duration:.2f}s")
            break
    
    if not processed_segments_info:
        print(f"No video segments could be successfully processed for {audio_file_path}. Skipping.")
        return

    # Recalculate total video duration from segment list accurately
    current_total_video_duration = sum(s_info['duration'] for s_info in processed_segments_info)
    print(f"  Total processed video segment duration: {current_total_video_duration:.2f}s (Audio: {audio_duration:.2f}s)")


    final_video_stream = None
    if len(processed_segments_info) == 1:
        final_video_stream = processed_segments_info[0]['stream']
    elif len(processed_segments_info) > 1:
        current_merged_stream = processed_segments_info[0]['stream']
        current_timeline_duration = processed_segments_info[0]['duration']

        for i in range(1, len(processed_segments_info)):
            next_segment_stream = processed_segments_info[i]['stream']
            next_segment_duration = processed_segments_info[i]['duration']
            
            # Only 'fade' transition will be used
            transition_type = XFADE_TRANSITIONS[0] 
            
            xfade_offset_val = current_timeline_duration - TRANSITION_DURATION 
            if xfade_offset_val < 0 : xfade_offset_val = 0 

            print(f"    Applying xfade ({transition_type}): dur={TRANSITION_DURATION:.2f}s, offset={xfade_offset_val:.2f}s (timeline at {current_timeline_duration:.2f}s)")
            
            try:
                current_merged_stream = ffmpeg.filter(
                    [current_merged_stream, next_segment_stream],
                    'xfade',
                    transition=transition_type,
                    duration=TRANSITION_DURATION,
                    offset=xfade_offset_val
                ).filter('setpts', 'PTS-STARTPTS')

                current_timeline_duration += (next_segment_duration - TRANSITION_DURATION)

            except ffmpeg.Error as e:
                print(f"    FFmpeg error during xfade ({transition_type}): {e.stderr.decode('utf8') if e.stderr else str(e)}")
                print("    Falling back to simple concatenation for the rest of this reel if xfade continues to fail.")
                remaining_streams_info = processed_segments_info[i:]
                
                # Create a list of streams from the point of failure
                streams_to_concat_fallback = [s_info['stream'] for s_info in remaining_streams_info]

                if current_merged_stream and streams_to_concat_fallback:
                     # Concatenate the stream that was successfully merged so far with the rest
                     all_streams_for_fallback = [current_merged_stream] + streams_to_concat_fallback
                     current_merged_stream = ffmpeg.concat(*all_streams_for_fallback, v=1, a=0).node 
                elif streams_to_concat_fallback: # If error happened on the very first xfade attempt
                     current_merged_stream = ffmpeg.concat(*streams_to_concat_fallback, v=1, a=0).node
                else: 
                    print("    Cannot fallback, no streams left after xfade error.")
                    return
                current_merged_stream = current_merged_stream.filter('setpts', 'PTS-STARTPTS')
                break 
        final_video_stream = current_merged_stream
    else: 
        return

    audio_input_stream = ffmpeg.input(audio_file_path).audio

    try:
        output_options = {
            'acodec': AUDIO_CODEC,
            'vcodec': VIDEO_CODEC,
            'audio_bitrate': AUDIO_BITRATE,
            'crf': CRF,
            'preset': PRESET_X264,
            'movflags': '+faststart', 
            'shortest': None, 
        }
        
        # Ensure the final stream has FPS set if it went through simple concat fallback
        # or if it's a single segment that didn't go through the xfade loop's final setpts.
        # The apply_segment_effects already applies fps and setpts.
        # However, a final global FPS can be a safeguard.
        # For simplicity, assuming apply_segment_effects handles it sufficiently.
        # If simple concat was used, it might need an explicit fps filter.
        if isinstance(final_video_stream, ffmpeg.nodes.FilterableStream) and \
           any(arg.key == 'concat' for arg in final_video_stream.node.args if hasattr(arg, 'key')): # Check if it was a concat node
            final_video_stream = final_video_stream.filter('fps', fps=OUTPUT_FPS, round='near')


        stream = ffmpeg.output(final_video_stream, audio_input_stream, output_file_path, **output_options)
        
        # print(f"  Final FFmpeg command args: {' '.join(ffmpeg.get_args(stream))}") # For debugging
        
        print(f"  Generating reel: {output_file_path}...")
        ffmpeg.run(stream, cmd='ffmpeg', overwrite_output=True, quiet=False)
        print(f"Successfully created Reel: {output_file_path}")

    except ffmpeg.Error as e:
        print(f"FFmpeg error during final output for {output_file_path}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
    except Exception as e:
        print(f"An unexpected error occurred during final output generation: {str(e)}")

def main():
    """
    Main function to find audio/video pairs and process them.
    """
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    
    script_root_dirs = [d for d in os.listdir(BASE_VIDEOS_PATH) if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script_folder_name in script_root_dirs:
        current_script_base_path = os.path.join(BASE_VIDEOS_PATH, script_folder_name)
        audio_dir_path = os.path.join(current_script_base_path, 'audio')
        visuals_dir_path = os.path.join(current_script_base_path, 'visuals')
        
        output_for_this_script = os.path.join(OUTPUT_DIR_BASE, script_folder_name)
        os.makedirs(output_for_this_script, exist_ok=True)

        if not os.path.isdir(audio_dir_path):
            print(f"Audio directory not found: {audio_dir_path}. Skipping script folder: {script_folder_name}")
            continue
        if not os.path.isdir(visuals_dir_path):
            print(f"Visuals directory not found: {visuals_dir_path}. Skipping script folder: {script_folder_name}")
            continue

        print(f"\nProcessing script folder: {script_folder_name}")
        audio_files = [f for f in os.listdir(audio_dir_path) if f.lower().endswith(('.wav', '.mp3', '.m4a', '.aac'))]

        for audio_filename in audio_files:
            audio_name_part = os.path.splitext(audio_filename)[0]
            full_audio_file_path = os.path.join(audio_dir_path, audio_filename)
            
            all_visual_files = os.listdir(visuals_dir_path)
            related_video_files = [
                os.path.join(visuals_dir_path, vf) 
                for vf in all_visual_files 
                if audio_name_part in os.path.splitext(vf)[0] and vf.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))
            ]
            
            if not related_video_files:
                print(f"  No videos specifically matching '{audio_name_part}' found. Trying all videos in '{visuals_dir_path}'.")
                related_video_files = [
                    os.path.join(visuals_dir_path, vf)
                    for vf in all_visual_files
                    if vf.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))
                ]
                if not related_video_files:
                    print(f"  No video files at all found in {visuals_dir_path} for audio {audio_filename}. Skipping.")
                    continue

            output_reel_filename = f"reel_{audio_name_part}.mp4"
            full_output_reel_path = os.path.join(output_for_this_script, output_reel_filename)

            create_reel_for_audio(full_audio_file_path, related_video_files, full_output_reel_path)

if __name__ == '__main__':
    main()
    print("\n--- Script Finished ---")


Processing script folder: The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect

Processing audio: REAL-WORLD_EXAMPLE.wav, Target Duration: 29.57s
  Segment from REAL-WORLD_EXAMPLE_8499763.mp4: ss=4.74s, t=5.00s
  Segment from REAL-WORLD_EXAMPLE_5426869.mp4: ss=7.40s, t=2.62s
  Segment from REAL-WORLD_EXAMPLE_8342244.mp4: ss=0.08s, t=2.98s
  Segment from REAL-WORLD_EXAMPLE_8499763.mp4: ss=6.05s, t=3.01s
  Segment from REAL-WORLD_EXAMPLE_5426869.mp4: ss=4.00s, t=4.01s
  Segment from REAL-WORLD_EXAMPLE_8342244.mp4: ss=1.97s, t=4.47s
  Segment from REAL-WORLD_EXAMPLE_8499763.mp4: ss=1.17s, t=4.50s
  Segment from REAL-WORLD_EXAMPLE_5426869.mp4: ss=16.61s, t=2.00s
  Segment from REAL-WORLD_EXAMPLE_8342244.mp4: ss=6.72s, t=0.99s
  Total processed video segment duration: 29.57s (Audio: 29.57s)
    Applying xfade (fade): dur=0.50s, offset=4.50s (timeline at 5.00s)
    Applying xfade (fade): dur=0.50s, offset=6.61s (timeline at 7.11s)
    Applying xfade (fade): dur=0.50s, offset=9

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Successfully created Reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_REAL-WORLD_EXAMPLE.mp4

Processing audio: CTA.wav, Target Duration: 6.23s
  Segment from CTA_8502588.mp4: ss=2.30s, t=3.85s
  Segment from CTA_9787494.mp4: ss=11.76s, t=2.38s
  Total processed video segment duration: 6.23s (Audio: 6.23s)
    Applying xfade (fade): dur=0.50s, offset=3.35s (timeline at 3.85s)
  Generating reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_CTA.mp4...


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Successfully created Reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_CTA.mp4

Processing audio: CONCEPT.wav, Target Duration: 11.67s
  Segment from CONCEPT_8531313.mp4: ss=3.51s, t=4.11s
  Segment from CONCEPT_8539651.mp4: ss=3.59s, t=3.74s
  Segment from CONCEPT_8852689.mp4: ss=0.17s, t=2.05s
  Segment from CONCEPT_8531313.mp4: ss=0.73s, t=1.78s
  Total processed video segment duration: 11.67s (Audio: 11.67s)
    Applying xfade (fade): dur=0.50s, offset=3.61s (timeline at 4.11s)
    Applying xfade (fade): dur=0.50s, offset=6.84s (timeline at 7.34s)
    Applying xfade (fade): dur=0.50s, offset=8.39s (timeline at 8.89s)
  Generating reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_CONCEPT.mp4...


...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Successfully created Reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_ACTIONABLE_TIP.mp4

Processing audio: HOOK.wav, Target Duration: 4.27s
  Segment from HOOK_6696210.mp4: ss=2.74s, t=3.84s
  Segment from HOOK_5124225.mp4: ss=7.42s, t=0.42s
  Total processed video segment duration: 4.27s (Audio: 4.27s)
    Applying xfade (fade): dur=0.50s, offset=3.34s (timeline at 3.84s)
  Generating reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_HOOK.mp4...


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Successfully created Reel: output_reels_fades_only/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/reel_HOOK.mp4

--- Script Finished ---


[mp4 @ 0x644272b72ac0] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x644272b72980] video:595kB audio:53kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.661201%
frame=  113 fps= 60 q=-1.0 Lsize=     652kB time=00:00:03.71 bitrate=1439.7kbits/s speed=1.98x    
[libx264 @ 0x644272b73b00] frame I:1     Avg QP:22.52  size: 49896
[libx264 @ 0x644272b73b00] frame P:33    Avg QP:22.92  size: 13030
[libx264 @ 0x644272b73b00] frame B:79    Avg QP:27.18  size:  1631
[libx264 @ 0x644272b73b00] consecutive B-frames:  0.9% 10.6% 21.2% 67.3%
[libx264 @ 0x644272b73b00] mb I  I16..4: 17.7% 76.8%  5.5%
[libx264 @ 0x644272b73b00] mb P  I16..4:  4.1%  9.4%  0.5%  P16..4: 41.9% 11.1%  4.0%  0.0%  0.0%    skip:29.0%
[libx264 @ 0x644272b73b00] mb B  I16..4:  0.1%  0.1%  0.0%  B16..8: 28.4%  0.6%  0.1%  direct: 0.3%  skip:70.5%  L0:50.1% L1:44.2% BI: 5.7%
[libx264 @ 0x644272b73b00] 8x8 transform intra:68.7% inter:84.5%
[libx264 @ 0x644272b73b00] 

In [ ]:
import ffmpeg
import random
import os
import glob
import shutil # For copying single files

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/"
OUTPUT_DIR_BASE = "output_reels_v3" 

# For individual segments within a reel
MIN_SEGMENT_DURATION_FALLBACK = 1.5 # Fallback if fixed duration is too short or videos are too short
TARGET_SEGMENTS_PER_AUDIO = 3 # How many video clips to feature per audio file

TRANSITION_DURATION = 0.5   
REEL_WIDTH = 720           
REEL_HEIGHT = 1280          
OUTPUT_FPS = 30             

VIDEO_CODEC = 'libx264'     
AUDIO_CODEC = 'aac'         
AUDIO_BITRATE = '128k'      
CRF = 23                    
PRESET_X264 = 'medium'      

# Restored diverse transitions for segments within a reel
XFADE_TRANSITIONS = [
    'fade', 'fadewhite', 'fadeblack',]

# Transition type between concatenated reels (can be a specific one or chosen randomly)
CONCAT_REEL_TRANSITION_TYPE = 'fade'
CONCAT_REEL_TRANSITION_DURATION = 0.7 # Slightly longer for between-reel transitions

# --- Helper Functions ---

def get_duration(filename):
    """Gets the duration of a media file."""
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except ffmpeg.Error as e:
        # print(f"FFmpeg error probing duration for {filename}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
        return None
    except Exception:
        # print(f"Error probing duration for {filename}: {str(e)}")
        return None

# --- Effect Application ---
def apply_segment_effects(video_node, segment_actual_duration, apply_fx: bool, video_path_for_log=""):
    """Applies scaling, cropping, Ken Burns, and color effects to a segment."""
    
    # 1. Scale to cover and crop to 9:16 (always applied)
    processed_node = video_node.filter('scale',
                                       w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                                       h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                                       eval='frame') 
    processed_node = processed_node.filter('crop',
                                           w=REEL_WIDTH,
                                           h=REEL_HEIGHT,
                                           x=f'(iw-{REEL_WIDTH})/2', 
                                           y=f'(ih-{REEL_HEIGHT})/2')

    if apply_fx:
        # 2. Ken Burns (Zoompan) - applied to ~70% of segments
        if random.random() < 0.7:
            zoom_amount = random.uniform(1.05, 1.20) 
            zoom_speed_factor = random.uniform(0.0008, 0.0012)
            
            zoompan_d_frames = int(segment_actual_duration * OUTPUT_FPS)
            if zoompan_d_frames >= OUTPUT_FPS * 0.5: 
                processed_node = processed_node.filter(
                    'zoompan',
                    z=f'min(zoom+{zoom_speed_factor},{zoom_amount})',
                    d=max(1, zoompan_d_frames), # duration in frames, must be at least 1
                    x=f'iw/2-(iw/zoom/2)',
                    y=f'ih/2-(ih/zoom/2)',
                    fps=OUTPUT_FPS,
                    s=f'{REEL_WIDTH}x{REEL_HEIGHT}'
                )

        # 3. Subtle Color/Mood Adjustments - applied to ~30% of segments
        if random.random() < 0.3:
            effect_choice = random.choice(['eq', 'vignette_subtle'])
            if effect_choice == 'eq':
                processed_node = processed_node.filter(
                    'eq',
                    contrast=random.uniform(1.0, 1.05),
                    saturation=random.uniform(1.0, 1.10)
                )
            elif effect_choice == 'vignette_subtle':
                processed_node = processed_node.filter('vignette', 
                                                       mode='backward', 
                                                       angle=f'PI/{random.uniform(5,8)}') # More subtle vignette
    
    # 4. Ensure consistent FPS for the segment and reset timestamps for concatenation (always applied)
    processed_node = processed_node.filter('fps', fps=OUTPUT_FPS, round='near')
    processed_node = processed_node.filter('format', 'yuv420p') 
    processed_node = processed_node.filter('setpts', 'PTS-STARTPTS')
    return processed_node

# --- Reel Creation for a Single Audio ---
def create_reel_for_audio(
    audio_file_path, 
    associated_video_files, 
    output_file_path,
    video_selection_mode: str = 'random', 
    num_segments_target: int = TARGET_SEGMENTS_PER_AUDIO,
    apply_visual_fx: bool = True,
    fx_transitions_list = None
):
    if fx_transitions_list is None:
        fx_transitions_list = XFADE_TRANSITIONS 

    audio_duration = get_duration(audio_file_path)
    if audio_duration is None or audio_duration <= 0:
        print(f"Invalid audio duration for {audio_file_path}. Skipping.")
        return
    if not associated_video_files:
        print(f"No video files found for audio: {audio_file_path}. Skipping.")
        return

    print(f"\nProcessing audio: {os.path.basename(audio_file_path)}, Target Duration: {audio_duration:.2f}s, Target Segments: {num_segments_target}")

    # 1. Prepare video list based on selection mode
    source_video_candidates = list(associated_video_files)
    if video_selection_mode == 'random':
        random.shuffle(source_video_candidates)
    elif video_selection_mode == 'order':
        source_video_candidates.sort() # Sort by name for consistent order if not random

    # 2. Determine fixed segment duration and select videos
    K = max(1, num_segments_target) # Ensure at least one segment
    fixed_segment_duration = audio_duration / K

    # Ensure segment duration is not excessively short (e.g., shorter than transition)
    min_practical_segment_duration = TRANSITION_DURATION + 0.2 # Must be longer than transition
    if K > 1 and fixed_segment_duration < min_practical_segment_duration:
        K_new = int(audio_duration / min_practical_segment_duration)
        K = max(1, K_new)
        fixed_segment_duration = audio_duration / K
        print(f"  Warning: Original num_segments_target resulted in too short segments. Adjusted to {K} segments.")
    
    if fixed_segment_duration <= 0:
        print(f"  Error: Calculated segment duration is zero or negative ({fixed_segment_duration:.2f}s). Skipping reel.")
        return

    # Filter for videos that are long enough and build the sequence of K videos
    eligible_videos = [v for v in source_video_candidates if (get_duration(v) or 0) >= fixed_segment_duration]
    
    selected_video_paths_for_segments = []
    if not eligible_videos:
        print(f"  Warning: No videos long enough for fixed segment duration ({fixed_segment_duration:.2f}s). Using any available videos; segments might be shorter.")
        if not source_video_candidates: print(f"  Error: No source videos at all for {audio_file_path}."); return
        # Fallback: use shorter videos, ffmpeg will take what it can up to 't'
        for i in range(K):
            selected_video_paths_for_segments.append(source_video_candidates[i % len(source_video_candidates)])
    else:
        for i in range(K):
            selected_video_paths_for_segments.append(eligible_videos[i % len(eligible_videos)])

    # 3. Create K segments
    processed_segments_info = []
    for i in range(K):
        video_path = selected_video_paths_for_segments[i]
        clip_native_duration = get_duration(video_path) or 0 # Default to 0 if None

        segment_t_param = fixed_segment_duration 
        
        max_ss = max(0, clip_native_duration - segment_t_param)
        start_time = random.uniform(0, max_ss) if max_ss > 0 else 0
        
        # If clip is shorter than segment_t_param, ffmpeg's 't' will effectively take the whole clip from 'ss'
        # The 'duration' for xfade calculations should still be the planned fixed_segment_duration.
        
        print(f"  Segment {i+1}/{K} from {os.path.basename(video_path)}: ss={start_time:.2f}s, planned_t={segment_t_param:.2f}s")

        try:
            input_node = ffmpeg.input(video_path, ss=start_time, t=segment_t_param)
            processed_video_stream = apply_segment_effects(input_node.video, segment_t_param, apply_visual_fx, video_path)
            processed_segments_info.append({'stream': processed_video_stream, 'duration': segment_t_param}) # Use planned duration for xfade
        except ffmpeg.Error as e:
            print(f"    FFmpeg error processing segment: {e.stderr.decode('utf8') if e.stderr else str(e)}")
            # Attempt to add a placeholder or skip, here we skip. Could be more robust.
            continue
        except Exception as e:
            print(f"    Unexpected error processing segment: {str(e)}")
            continue
            
    if not processed_segments_info:
        print(f"  No video segments successfully processed for {audio_file_path}. Skipping reel.")
        return

    # 4. Concatenate segments with xfade
    final_video_stream = None
    if len(processed_segments_info) == 1:
        final_video_stream = processed_segments_info[0]['stream']
    elif len(processed_segments_info) > 1:
        current_merged_stream = processed_segments_info[0]['stream']
        current_timeline_duration = processed_segments_info[0]['duration'] # Planned duration

        for i in range(1, len(processed_segments_info)):
            next_segment_stream = processed_segments_info[i]['stream']
            next_segment_duration = processed_segments_info[i]['duration'] # Planned duration
            
            transition_type = random.choice(fx_transitions_list)
            xfade_offset_val = current_timeline_duration - TRANSITION_DURATION 
            if xfade_offset_val < 0: xfade_offset_val = 0 

            print(f"    Applying xfade ({transition_type}) between segments: dur={TRANSITION_DURATION:.2f}s, offset={xfade_offset_val:.2f}s")
            try:
                current_merged_stream = ffmpeg.filter(
                    [current_merged_stream, next_segment_stream],
                    'xfade', transition=transition_type, duration=TRANSITION_DURATION, offset=xfade_offset_val
                ).filter('setpts', 'PTS-STARTPTS')
                current_timeline_duration += (next_segment_duration - TRANSITION_DURATION)
            except ffmpeg.Error as e:
                # Fallback logic from previous version
                print(f"    FFmpeg error during xfade: {e.stderr.decode('utf8') if e.stderr else str(e)}. Falling back.");
                remaining_streams_info = processed_segments_info[i:]
                streams_to_concat_fallback = [s_info['stream'] for s_info in remaining_streams_info]
                if current_merged_stream and streams_to_concat_fallback:
                     all_streams_for_fallback = [current_merged_stream] + streams_to_concat_fallback
                     current_merged_stream = ffmpeg.concat(*all_streams_for_fallback, v=1, a=0).node[0]
                elif streams_to_concat_fallback:
                     current_merged_stream = ffmpeg.concat(*streams_to_concat_fallback, v=1, a=0).node[0]
                else: print("    Cannot fallback."); return
                current_merged_stream = current_merged_stream.filter('setpts', 'PTS-STARTPTS')
                break 
        final_video_stream = current_merged_stream
    
    if final_video_stream is None: print(f"  Failed to create final video stream for {audio_file_path}."); return

    # 5. Final Output
    audio_input_stream = ffmpeg.input(audio_file_path).audio
    try:
        output_options = {
            'acodec': AUDIO_CODEC, 'vcodec': VIDEO_CODEC, 'audio_bitrate': AUDIO_BITRATE,
            'crf': CRF, 'preset': PRESET_X264, 'movflags': '+faststart', 'shortest': None,
        }
        # Ensure final FPS, especially if concat fallback was used.
        final_video_stream = final_video_stream.filter('fps', fps=OUTPUT_FPS, round='near')

        stream = ffmpeg.output(final_video_stream, audio_input_stream, output_file_path, **output_options)
        print(f"  Generating reel: {os.path.basename(output_file_path)}...")
        ffmpeg.run(stream, cmd='ffmpeg', overwrite_output=True, quiet=True) # Set quiet=False for debug
        print(f"  Successfully created reel: {output_file_path}")
    except ffmpeg.Error as e:
        print(f"  FFmpeg error during final output for {output_file_path}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
    except Exception as e:
        print(f"  Unexpected error for {output_file_path}: {str(e)}")


# --- Concatenation of Multiple Reels ---
def concatenate_reels(
    reel_paths: list,
    final_output_path: str,
    master_fps: int = OUTPUT_FPS, # Use global FPS
    transition_type: str = CONCAT_REEL_TRANSITION_TYPE,
    transition_duration: float = CONCAT_REEL_TRANSITION_DURATION
):
    if not reel_paths:
        print("No reels to concatenate.")
        return
    if len(reel_paths) == 1:
        try:
            if os.path.abspath(reel_paths[0]) != os.path.abspath(final_output_path): # Avoid copying to self
                 shutil.copy2(reel_paths[0], final_output_path) # Use copy2 to preserve metadata
                 print(f"Only one reel, copied to {final_output_path}")
            else:
                 print(f"Single reel is already the target output: {final_output_path}")

        except Exception as e:
            print(f"Error copying single reel {reel_paths[0]} to {final_output_path}: {e}")
        return

    print(f"\nConcatenating {len(reel_paths)} reels into {os.path.basename(final_output_path)}...")

    input_streams_info = []
    for i, path in enumerate(reel_paths):
        duration = get_duration(path)
        if duration is None or duration <=0:
            print(f"  Warning: Could not get valid duration for reel {path}. Skipping.")
            continue

        video_input = ffmpeg.input(path).video.filter('fps', fps=master_fps, round='near').filter('setpts', 'PTS-STARTPTS')
        audio_input = ffmpeg.input(path).audio.filter('aresample', **{'async': 1, 'first_pts': 0})# Resample and sync audio

        input_streams_info.append({
            'video': video_input,
            'audio': audio_input,
            'duration': duration, # Original duration for xfade offset calculation
            'path': path
        })

    if not input_streams_info:
        print("  No valid reel streams to concatenate after filtering.")
        return
    if len(input_streams_info) == 1: # If only one valid reel left
        try:
            if os.path.abspath(input_streams_info[0]['path']) != os.path.abspath(final_output_path):
                shutil.copy2(input_streams_info[0]['path'], final_output_path)
                print(f"Only one valid reel, copied to {final_output_path}")
            else:
                print(f"Single valid reel is already the target output: {final_output_path}")
        except Exception as e:
            print(f"Error copying single valid reel: {e}")
        return


    # Video concatenation with xfade
    current_merged_video = input_streams_info[0]['video']
    current_video_timeline_duration = input_streams_info[0]['duration']

    for i in range(1, len(input_streams_info)):
        next_video_stream = input_streams_info[i]['video']
        next_video_duration = input_streams_info[i]['duration']

        xfade_offset_val = current_video_timeline_duration - transition_duration
        if xfade_offset_val < 0: xfade_offset_val = 0

        print(f"  Concatenating with xfade ({transition_type}) between reels: dur={transition_duration:.2f}s, offset={xfade_offset_val:.2f}s")
        try:
            current_merged_video = ffmpeg.filter(
                [current_merged_video, next_video_stream],
                'xfade', transition=transition_type, duration=transition_duration, offset=xfade_offset_val
            ).filter('setpts', 'PTS-STARTPTS')
            current_video_timeline_duration += (next_video_duration - transition_duration)
        except ffmpeg.Error as e:
            print(f"    FFmpeg error during reel xfade: {e.stderr.decode('utf8') if e.stderr else str(e)}. Aborting concatenation.")
            return

    # Audio concatenation
    audio_streams = [info['audio'] for info in input_streams_info]
    concatenated_audio = None
    if audio_streams:
        concatenated_audio = ffmpeg.concat(*audio_streams, v=0, a=1) # Removed .stream here


    try:
        output_options = {
            'acodec': AUDIO_CODEC, 'vcodec': VIDEO_CODEC, 'audio_bitrate': AUDIO_BITRATE,
            'crf': CRF, 'preset': PRESET_X264, 'movflags': '+faststart', 'shortest': None
        }
        stream = ffmpeg.output(current_merged_video, concatenated_audio, final_output_path, **output_options)
        print(f"  Generating final concatenated video: {os.path.basename(final_output_path)}...")
        ffmpeg.run(stream, cmd='ffmpeg', overwrite_output=True, quiet=True) # Set quiet=False for debug
        print(f"  Successfully concatenated reels into: {final_output_path}")
    except ffmpeg.Error as e:
        print(f"  FFmpeg error during final reel concatenation for {final_output_path}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
    except Exception as e:
        print(f"  Unexpected error for {final_output_path}: {str(e)}")

# --- Main Processing Logic ---
def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    script_root_dirs = [d for d in os.listdir(BASE_VIDEOS_PATH) if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script_folder_name in script_root_dirs:
        current_script_base_path = os.path.join(BASE_VIDEOS_PATH, script_folder_name)
        audio_dir_path = os.path.join(current_script_base_path, 'audio')
        visuals_dir_path = os.path.join(current_script_base_path, 'visuals')

        output_script_folder = os.path.join(OUTPUT_DIR_BASE, script_folder_name)
        os.makedirs(output_script_folder, exist_ok=True)

        # Sub-directory for individual reels before final concatenation
        individual_reels_temp_dir = os.path.join(output_script_folder, "individual_reels")
        os.makedirs(individual_reels_temp_dir, exist_ok=True)


        if not (os.path.isdir(audio_dir_path) and os.path.isdir(visuals_dir_path)):
            print(f"Audio or visuals directory missing in {script_folder_name}. Skipping.")
            continue

        print(f"\n>>> Processing Script Folder: {script_folder_name}")
        audio_files = sorted([f for f in os.listdir(audio_dir_path) if f.lower().endswith(('.wav', '.mp3', '.m4a', '.aac'))])

        created_reel_paths_for_this_script_folder = []

        for audio_filename in audio_files:
            audio_name_part = os.path.splitext(audio_filename)[0]
            full_audio_file_path = os.path.join(audio_dir_path, audio_filename)

            all_visual_files = os.listdir(visuals_dir_path)
            related_video_files = sorted([ # Sort for 'order' mode predictability
                os.path.join(visuals_dir_path, vf)
                for vf in all_visual_files
                if audio_name_part in os.path.splitext(vf)[0] and vf.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))
            ])

            if not related_video_files:
                print(f"  No videos specifically matching '{audio_name_part}'. Using all videos in '{visuals_dir_path}'.")
                related_video_files = sorted([
                    os.path.join(visuals_dir_path, vf)
                    for vf in all_visual_files
                    if vf.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))
                ])
                if not related_video_files:
                    print(f"  No video files found in {visuals_dir_path} for audio {audio_filename}. Skipping this audio.")
                    continue

            # Output path for the individual reel for this audio file
            # Place it in a temporary subfolder for better organization
            temp_output_reel_filename = f"reel_{audio_name_part}.mp4"
            temp_full_output_reel_path = os.path.join(individual_reels_temp_dir, temp_output_reel_filename)

            # --- Parameters for create_reel_for_audio ---
            # You can customize these per audio type if needed, or make them global settings
            current_video_selection_mode = 'random' # 'order' or 'random'
            current_num_segments = TARGET_SEGMENTS_PER_AUDIO
            current_apply_fx = True

            create_reel_for_audio(
                full_audio_file_path,
                related_video_files,
                temp_full_output_reel_path,
                video_selection_mode=current_video_selection_mode,
                num_segments_target=current_num_segments,
                apply_visual_fx=current_apply_fx
            )
            # Check if the reel was actually created before adding to list
            if os.path.exists(temp_full_output_reel_path) and os.path.getsize(temp_full_output_reel_path) > 0:
                 created_reel_paths_for_this_script_folder.append(temp_full_output_reel_path)
            else:
                 print(f"  Warning: Reel {temp_full_output_reel_path} was not created or is empty.")


        # After processing all audios in the current script folder, concatenate the created reels
        if created_reel_paths_for_this_script_folder:
            final_video_filename = f"FINAL_VIDEO_{script_folder_name.replace(' ', '_')}.mp4"
            final_output_path_for_script_folder = os.path.join(output_script_folder, final_video_filename)

            concatenate_reels(
                created_reel_paths_for_this_script_folder,
                final_output_path_for_script_folder,
                transition_type=CONCAT_REEL_TRANSITION_TYPE,
                transition_duration=CONCAT_REEL_TRANSITION_DURATION
            )
        else:
            print(f"No individual reels were created for script folder: {script_folder_name}. Skipping final concatenation.")


if __name__ == '__main__':
    main()
    print("\n--- Script Finished ---")



>>> Processing Script Folder: The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect

Processing audio: ACTIONABLE_TIP.wav, Target Duration: 11.31s, Target Segments: 3
  Segment 1/3 from ACTIONABLE_TIP_8639847.mp4: ss=4.49s, planned_t=3.77s
  Segment 2/3 from ACTIONABLE_TIP_4460555.mp4: ss=4.47s, planned_t=3.77s
  Segment 3/3 from ACTIONABLE_TIP_13736550.mp4: ss=3.53s, planned_t=3.77s
    Applying xfade (fade) between segments: dur=0.50s, offset=3.27s
    Applying xfade (dissolve) between segments: dur=0.50s, offset=6.54s
  Generating reel: reel_ACTIONABLE_TIP.mp4...
  Successfully created reel: output_reels_v3/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/individual_reels/reel_ACTIONABLE_TIP.mp4

Processing audio: CONCEPT.wav, Target Duration: 11.67s, Target Segments: 3
  Segment 1/3 from CONCEPT_8852689.mp4: ss=8.85s, planned_t=3.89s
  Segment 2/3 from CONCEPT_8531313.mp4: ss=2.87s, planned_t=3.89s
  Segment 3/3 from CONCEPT_8539651.mp4: ss=8.42s, pl

In [6]:
help(ffmpeg.filter('aresample'))

TypeError: filter() missing 1 required positional argument: 'filter_name'

In [7]:
help(ffmpeg.filter)

Help on function filter in module ffmpeg._filters:

filter(stream_spec, filter_name, *args, **kwargs)
    Apply custom filter.

    ``filter_`` is normally used by higher-level filter functions such as ``hflip``, but if a filter implementation
    is missing from ``ffmpeg-python``, you can call ``filter_`` directly to have ``ffmpeg-python`` pass the filter name
    and arguments to ffmpeg verbatim.

    Args:
        stream_spec: a Stream, list of Streams, or label-to-Stream dictionary mapping
        filter_name: ffmpeg filter name, e.g. `colorchannelmixer`
        *args: list of args to pass to ffmpeg verbatim
        **kwargs: list of keyword-args to pass to ffmpeg verbatim

    The function name is suffixed with ``_`` in order avoid confusion with the standard python ``filter`` function.

    Example:

        ``ffmpeg.input('in.mp4').filter('hflip').output('out.mp4').run()``



In [ ]:
import ffmpeg
import random
import os
import glob
import shutil # For copying single files

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/"
OUTPUT_DIR_BASE = "output_reels_v3" 

# For individual segments within a reel
MIN_SEGMENT_DURATION_FALLBACK = 1.5 # Fallback if fixed duration is too short or videos are too short
TARGET_SEGMENTS_PER_AUDIO = 3 # How many video clips to feature per audio file

TRANSITION_DURATION = 0.5   
REEL_WIDTH = 720           
REEL_HEIGHT = 1280          
OUTPUT_FPS = 30             

VIDEO_CODEC = 'libx264'     
AUDIO_CODEC = 'aac'         
AUDIO_BITRATE = '128k'      
CRF = 23                    
PRESET_X264 = 'medium'      

# Restored diverse transitions for segments within a reel
XFADE_TRANSITIONS = ['fade']  # Between segments within a reel

# For final concatenation of reels
CONCAT_REEL_TRANSITION_TYPE = 'fade'
CONCAT_REEL_TRANSITION_DURATION = 0.7 # Slightly longer for between-reel transitions

# --- Helper Functions ---

def get_duration(filename):
    """Gets the duration of a media file."""
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except ffmpeg.Error as e:
        # print(f"FFmpeg error probing duration for {filename}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
        return None
    except Exception:
        # print(f"Error probing duration for {filename}: {str(e)}")
        return None

# --- Effect Application ---
def apply_segment_effects(video_node, segment_actual_duration, apply_fx: bool, video_path_for_log=""):
    """Minimal processing: scale & crop to 9:16 and set FPS. No visual effects."""
    
    # 1. Scale and crop to 9:16
    processed_node = video_node.filter('scale',
                                       w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                                       h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                                       eval='frame')
    processed_node = processed_node.filter('crop',
                                           w=REEL_WIDTH,
                                           h=REEL_HEIGHT,
                                           x=f'(iw-{REEL_WIDTH})/2',
                                           y=f'(ih-{REEL_HEIGHT})/2')

    # 2. Set FPS and format
    processed_node = processed_node.filter('fps', fps=OUTPUT_FPS, round='near')
    processed_node = processed_node.filter('format', 'yuv420p')
    processed_node = processed_node.filter('setpts', 'PTS-STARTPTS')
    return processed_node


# --- Reel Creation for a Single Audio ---
def create_reel_for_audio(
    audio_file_path, 
    associated_video_files, 
    output_file_path,
    video_selection_mode: str = 'random', 
    num_segments_target: int = TARGET_SEGMENTS_PER_AUDIO,
    apply_visual_fx: bool = True,
    fx_transitions_list = None
):
    if fx_transitions_list is None:
        fx_transitions_list = XFADE_TRANSITIONS 

    audio_duration = get_duration(audio_file_path)
    if audio_duration is None or audio_duration <= 0:
        print(f"Invalid audio duration for {audio_file_path}. Skipping.")
        return
    if not associated_video_files:
        print(f"No video files found for audio: {audio_file_path}. Skipping.")
        return

    print(f"\nProcessing audio: {os.path.basename(audio_file_path)}, Target Duration: {audio_duration:.2f}s, Target Segments: {num_segments_target}")

    # 1. Prepare video list based on selection mode
    source_video_candidates = list(associated_video_files)
    if video_selection_mode == 'random':
        random.shuffle(source_video_candidates)
    elif video_selection_mode == 'order':
        source_video_candidates.sort() # Sort by name for consistent order if not random

    # 2. Determine fixed segment duration and select videos
    K = max(1, num_segments_target) # Ensure at least one segment
    fixed_segment_duration = audio_duration / K

    # Ensure segment duration is not excessively short (e.g., shorter than transition)
    min_practical_segment_duration = TRANSITION_DURATION + 0.2 # Must be longer than transition
    if K > 1 and fixed_segment_duration < min_practical_segment_duration:
        K_new = int(audio_duration / min_practical_segment_duration)
        K = max(1, K_new)
        fixed_segment_duration = audio_duration / K
        print(f"  Warning: Original num_segments_target resulted in too short segments. Adjusted to {K} segments.")
    
    if fixed_segment_duration <= 0:
        print(f"  Error: Calculated segment duration is zero or negative ({fixed_segment_duration:.2f}s). Skipping reel.")
        return

    # Filter for videos that are long enough and build the sequence of K videos
    eligible_videos = [v for v in source_video_candidates if (get_duration(v) or 0) >= fixed_segment_duration]
    
    selected_video_paths_for_segments = []
    if not eligible_videos:
        print(f"  Warning: No videos long enough for fixed segment duration ({fixed_segment_duration:.2f}s). Using any available videos; segments might be shorter.")
        if not source_video_candidates: print(f"  Error: No source videos at all for {audio_file_path}."); return
        # Fallback: use shorter videos, ffmpeg will take what it can up to 't'
        for i in range(K):
            selected_video_paths_for_segments.append(source_video_candidates[i % len(source_video_candidates)])
    else:
        for i in range(K):
            selected_video_paths_for_segments.append(eligible_videos[i % len(eligible_videos)])

    # 3. Create K segments
    processed_segments_info = []
    for i in range(K):
        video_path = selected_video_paths_for_segments[i]
        clip_native_duration = get_duration(video_path) or 0 # Default to 0 if None

        segment_t_param = fixed_segment_duration 
        
        max_ss = max(0, clip_native_duration - segment_t_param)
        start_time = random.uniform(0, max_ss) if max_ss > 0 else 0
        
        # If clip is shorter than segment_t_param, ffmpeg's 't' will effectively take the whole clip from 'ss'
        # The 'duration' for xfade calculations should still be the planned fixed_segment_duration.
        
        print(f"  Segment {i+1}/{K} from {os.path.basename(video_path)}: ss={start_time:.2f}s, planned_t={segment_t_param:.2f}s")

        try:
            input_node = ffmpeg.input(video_path, ss=start_time, t=segment_t_param)
            processed_video_stream = apply_segment_effects(input_node.video, segment_t_param, apply_visual_fx, video_path)
            processed_segments_info.append({'stream': processed_video_stream, 'duration': segment_t_param}) # Use planned duration for xfade
        except ffmpeg.Error as e:
            print(f"    FFmpeg error processing segment: {e.stderr.decode('utf8') if e.stderr else str(e)}")
            # Attempt to add a placeholder or skip, here we skip. Could be more robust.
            continue
        except Exception as e:
            print(f"    Unexpected error processing segment: {str(e)}")
            continue
            
    if not processed_segments_info:
        print(f"  No video segments successfully processed for {audio_file_path}. Skipping reel.")
        return

    # 4. Concatenate segments with xfade
    final_video_stream = None
    if len(processed_segments_info) == 1:
        final_video_stream = processed_segments_info[0]['stream']
    elif len(processed_segments_info) > 1:
        current_merged_stream = processed_segments_info[0]['stream']
        current_timeline_duration = processed_segments_info[0]['duration'] # Planned duration

        for i in range(1, len(processed_segments_info)):
            next_segment_stream = processed_segments_info[i]['stream']
            next_segment_duration = processed_segments_info[i]['duration'] # Planned duration
            
            transition_type = random.choice(fx_transitions_list)
            xfade_offset_val = current_timeline_duration - TRANSITION_DURATION 
            if xfade_offset_val < 0: xfade_offset_val = 0 

            print(f"    Applying xfade ({transition_type}) between segments: dur={TRANSITION_DURATION:.2f}s, offset={xfade_offset_val:.2f}s")
            try:
                current_merged_stream = ffmpeg.filter(
                    [current_merged_stream, next_segment_stream],
                    'xfade', transition=transition_type, duration=TRANSITION_DURATION, offset=xfade_offset_val
                ).filter('setpts', 'PTS-STARTPTS')
                current_timeline_duration += (next_segment_duration - TRANSITION_DURATION)
            except ffmpeg.Error as e:
                # Fallback logic from previous version
                print(f"    FFmpeg error during xfade: {e.stderr.decode('utf8') if e.stderr else str(e)}. Falling back.");
                remaining_streams_info = processed_segments_info[i:]
                streams_to_concat_fallback = [s_info['stream'] for s_info in remaining_streams_info]
                if current_merged_stream and streams_to_concat_fallback:
                     all_streams_for_fallback = [current_merged_stream] + streams_to_concat_fallback
                     current_merged_stream = ffmpeg.concat(*all_streams_for_fallback, v=1, a=0).node[0]
                elif streams_to_concat_fallback:
                     current_merged_stream = ffmpeg.concat(*streams_to_concat_fallback, v=1, a=0).node[0]
                else: print("    Cannot fallback."); return
                current_merged_stream = current_merged_stream.filter('setpts', 'PTS-STARTPTS')
                break 
        final_video_stream = current_merged_stream
    
    if final_video_stream is None: print(f"  Failed to create final video stream for {audio_file_path}."); return

    # 5. Final Output
    audio_input_stream = ffmpeg.input(audio_file_path).audio
    try:
        output_options = {
            'acodec': AUDIO_CODEC, 'vcodec': VIDEO_CODEC, 'audio_bitrate': AUDIO_BITRATE,
            'crf': CRF, 'preset': PRESET_X264, 'movflags': '+faststart', 'shortest': None,
        }
        # Ensure final FPS, especially if concat fallback was used.
        final_video_stream = final_video_stream.filter('fps', fps=OUTPUT_FPS, round='near')

        stream = ffmpeg.output(final_video_stream, audio_input_stream, output_file_path, **output_options)
        print(f"  Generating reel: {os.path.basename(output_file_path)}...")
        ffmpeg.run(stream, cmd='ffmpeg', overwrite_output=True, quiet=True) # Set quiet=False for debug
        print(f"  Successfully created reel: {output_file_path}")
    except ffmpeg.Error as e:
        print(f"  FFmpeg error during final output for {output_file_path}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
    except Exception as e:
        print(f"  Unexpected error for {output_file_path}: {str(e)}")


# --- Concatenation of Multiple Reels ---
def concatenate_reels(
    reel_paths: list,
    final_output_path: str,
    master_fps: int = OUTPUT_FPS, # Use global FPS
    transition_type: str = CONCAT_REEL_TRANSITION_TYPE,
    transition_duration: float = CONCAT_REEL_TRANSITION_DURATION
):
    if not reel_paths:
        print("No reels to concatenate.")
        return
    if len(reel_paths) == 1:
        try:
            if os.path.abspath(reel_paths[0]) != os.path.abspath(final_output_path): # Avoid copying to self
                 shutil.copy2(reel_paths[0], final_output_path) # Use copy2 to preserve metadata
                 print(f"Only one reel, copied to {final_output_path}")
            else:
                 print(f"Single reel is already the target output: {final_output_path}")

        except Exception as e:
            print(f"Error copying single reel {reel_paths[0]} to {final_output_path}: {e}")
        return

    print(f"\nConcatenating {len(reel_paths)} reels into {os.path.basename(final_output_path)}...")

    input_streams_info = []
    for i, path in enumerate(reel_paths):
        duration = get_duration(path)
        if duration is None or duration <=0:
            print(f"  Warning: Could not get valid duration for reel {path}. Skipping.")
            continue

        video_input = ffmpeg.input(path).video.filter('fps', fps=master_fps, round='near').filter('setpts', 'PTS-STARTPTS')
        audio_input = ffmpeg.input(path).audio.filter('aresample', **{'async': 1, 'first_pts': 0})# Resample and sync audio

        input_streams_info.append({
            'video': video_input,
            'audio': audio_input,
            'duration': duration, # Original duration for xfade offset calculation
            'path': path
        })

    if not input_streams_info:
        print("  No valid reel streams to concatenate after filtering.")
        return
    if len(input_streams_info) == 1: # If only one valid reel left
        try:
            if os.path.abspath(input_streams_info[0]['path']) != os.path.abspath(final_output_path):
                shutil.copy2(input_streams_info[0]['path'], final_output_path)
                print(f"Only one valid reel, copied to {final_output_path}")
            else:
                print(f"Single valid reel is already the target output: {final_output_path}")
        except Exception as e:
            print(f"Error copying single valid reel: {e}")
        return


    # Video concatenation with xfade
    current_merged_video = input_streams_info[0]['video']
    current_video_timeline_duration = input_streams_info[0]['duration']

    for i in range(1, len(input_streams_info)):
        next_video_stream = input_streams_info[i]['video']
        next_video_duration = input_streams_info[i]['duration']

        xfade_offset_val = current_video_timeline_duration - transition_duration
        if xfade_offset_val < 0: xfade_offset_val = 0

        print(f"  Concatenating with xfade ({transition_type}) between reels: dur={transition_duration:.2f}s, offset={xfade_offset_val:.2f}s")
        try:
            current_merged_video = ffmpeg.filter(
                [current_merged_video, next_video_stream],
                'xfade', transition=transition_type, duration=transition_duration, offset=xfade_offset_val
            ).filter('setpts', 'PTS-STARTPTS')
            current_video_timeline_duration += (next_video_duration - transition_duration)
        except ffmpeg.Error as e:
            print(f"    FFmpeg error during reel xfade: {e.stderr.decode('utf8') if e.stderr else str(e)}. Aborting concatenation.")
            return

    # Audio concatenation
    audio_streams = [info['audio'] for info in input_streams_info]
    concatenated_audio = None
    if audio_streams:
        concatenated_audio = ffmpeg.concat(*audio_streams, v=0, a=1) # Removed .stream here


    try:
        output_options = {
            'acodec': AUDIO_CODEC, 'vcodec': VIDEO_CODEC, 'audio_bitrate': AUDIO_BITRATE,
            'crf': CRF, 'preset': PRESET_X264, 'movflags': '+faststart', 'shortest': None
        }
        stream = ffmpeg.output(current_merged_video, concatenated_audio, final_output_path, **output_options)
        print(f"  Generating final concatenated video: {os.path.basename(final_output_path)}...")
        ffmpeg.run(stream, cmd='ffmpeg', overwrite_output=True, quiet=True) # Set quiet=False for debug
        print(f"  Successfully concatenated reels into: {final_output_path}")
    except ffmpeg.Error as e:
        print(f"  FFmpeg error during final reel concatenation for {final_output_path}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
    except Exception as e:
        print(f"  Unexpected error for {final_output_path}: {str(e)}")



SEGMENT_ORDER = [
    'HOOK',
    'CONCEPT',
    'REAL-WORLD_EXAMPLE',  # No hyphen here, use underscore
    'PSYCHOLOGICAL_INSIGHT',  # No hyphen here, use underscore
    'ACTIONABLE_TIP',  # No hyphen here, use underscore
    'CTA'
]



def concatenate_reels_in_custom_order(individual_reel_dir, final_output_path):
    if not os.path.isdir(individual_reel_dir):
        print(f"Directory not found: {individual_reel_dir}")
        return

    # Find all reel files in the directory
    all_reels = glob.glob(os.path.join(individual_reel_dir, "*.mp4"))
    if not all_reels:
        print(f"No individual reels found in {individual_reel_dir}")
        return

    # Sort according to SEGMENT_ORDER
    ordered_reels = []
    for keyword in SEGMENT_ORDER:
        for reel in all_reels:
            if keyword in os.path.basename(reel):  # Case-sensitive matching
                ordered_reels.append(reel)
                break  # Move to next keyword once found

    # Check if all segments were found
    missing_segments = [k for k in SEGMENT_ORDER if not any(k in os.path.basename(r) for r in ordered_reels)]
    if missing_segments:
        print(f"Missing segments: {', '.join(missing_segments)}. Cannot proceed with concatenation.")
        return

    print(f"Concatenating reels in this order:")
    for reel in ordered_reels:
        print(f"   - {os.path.basename(reel)}")

    # Perform concatenation using existing function
    concatenate_reels(
        reel_paths=ordered_reels,
        final_output_path=final_output_path,
        transition_type=CONCAT_REEL_TRANSITION_TYPE,
        transition_duration=CONCAT_REEL_TRANSITION_DURATION
    )


# --- Main Processing Logic ---
def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    script_root_dirs = [d for d in os.listdir(BASE_VIDEOS_PATH) if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script_folder_name in script_root_dirs:
        current_script_base_path = os.path.join(BASE_VIDEOS_PATH, script_folder_name)
        audio_dir_path = os.path.join(current_script_base_path, 'audio')
        visuals_dir_path = os.path.join(current_script_base_path, 'visuals')

        output_script_folder = os.path.join(OUTPUT_DIR_BASE, script_folder_name)
        os.makedirs(output_script_folder, exist_ok=True)

        # Sub-directory for individual reels before final concatenation
        individual_reels_temp_dir = os.path.join(output_script_folder, "individual_reels")
        os.makedirs(individual_reels_temp_dir, exist_ok=True)


        if not (os.path.isdir(audio_dir_path) and os.path.isdir(visuals_dir_path)):
            print(f"Audio or visuals directory missing in {script_folder_name}. Skipping.")
            continue

        print(f"\n>>> Processing Script Folder: {script_folder_name}")
        audio_files = sorted([f for f in os.listdir(audio_dir_path) if f.lower().endswith(('.wav', '.mp3', '.m4a', '.aac'))])

        created_reel_paths_for_this_script_folder = []

        for audio_filename in audio_files:
            audio_name_part = os.path.splitext(audio_filename)[0]
            full_audio_file_path = os.path.join(audio_dir_path, audio_filename)

            all_visual_files = os.listdir(visuals_dir_path)
            related_video_files = sorted([ # Sort for 'order' mode predictability
                os.path.join(visuals_dir_path, vf)
                for vf in all_visual_files
                if audio_name_part in os.path.splitext(vf)[0] and vf.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))
            ])

            if not related_video_files:
                print(f"  No videos specifically matching '{audio_name_part}'. Using all videos in '{visuals_dir_path}'.")
                related_video_files = sorted([
                    os.path.join(visuals_dir_path, vf)
                    for vf in all_visual_files
                    if vf.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))
                ])
                if not related_video_files:
                    print(f"  No video files found in {visuals_dir_path} for audio {audio_filename}. Skipping this audio.")
                    continue

            # Output path for the individual reel for this audio file
            # Place it in a temporary subfolder for better organization
            temp_output_reel_filename = f"reel_{audio_name_part}.mp4"
            temp_full_output_reel_path = os.path.join(individual_reels_temp_dir, temp_output_reel_filename)

            # --- Parameters for create_reel_for_audio ---
            # You can customize these per audio type if needed, or make them global settings
            current_video_selection_mode = 'random' # 'order' or 'random'
            current_num_segments = TARGET_SEGMENTS_PER_AUDIO
            current_apply_fx = True

            create_reel_for_audio(
                full_audio_file_path,
                related_video_files,
                temp_full_output_reel_path,
                video_selection_mode=current_video_selection_mode,
                num_segments_target=current_num_segments,
                apply_visual_fx=current_apply_fx
            )
            # Check if the reel was actually created before adding to list
            if os.path.exists(temp_full_output_reel_path) and os.path.getsize(temp_full_output_reel_path) > 0:
                 created_reel_paths_for_this_script_folder.append(temp_full_output_reel_path)
            else:
                 print(f"  Warning: Reel {temp_full_output_reel_path} was not created or is empty.")


        # After processing all audios in the current script folder, concatenate the created reels
        if created_reel_paths_for_this_script_folder:
            final_video_filename = f"FINAL_VIDEO_{script_folder_name.replace(' ', '_')}.mp4"
            final_output_path_for_script_folder = os.path.join(output_script_folder, final_video_filename)

            concatenate_reels_in_custom_order(
                individual_reels_temp_dir,
                final_output_path_for_script_folder
            )

        else:
            print(f"No individual reels were created for script folder: {script_folder_name}. Skipping final concatenation.")


if __name__ == '__main__':
    main()
    print("\n--- Script Finished ---")



>>> Processing Script Folder: The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect

Processing audio: ACTIONABLE_TIP.wav, Target Duration: 11.31s, Target Segments: 3
  Segment 1/3 from ACTIONABLE_TIP_13736550.mp4: ss=7.72s, planned_t=3.77s
    Unexpected error processing segment: Expected string index (e.g. 'a'); got 0
  Segment 2/3 from ACTIONABLE_TIP_4460555.mp4: ss=5.10s, planned_t=3.77s
    Unexpected error processing segment: Expected string index (e.g. 'a'); got 0
  Segment 3/3 from ACTIONABLE_TIP_8639847.mp4: ss=13.03s, planned_t=3.77s
    Unexpected error processing segment: Expected string index (e.g. 'a'); got 0
  No video segments successfully processed for /home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/audio/ACTIONABLE_TIP.wav. Skipping reel.

Processing audio: CONCEPT.wav, Target Duration: 11.67s, Target Segments: 3
  Segment 1/3 from CONCEPT_8852689.mp4: ss=8.07s, 

In [4]:
os.listdir('/home/aditya-ladawa/Aditya/z_projects/short_creation/src/react_agent/output_reels_v3/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/individual_reels')

['reel_ACTIONABLE_TIP.mp4',
 'reel_REAL-WORLD_EXAMPLE.mp4',
 'reel_HOOK.mp4',
 'reel_CTA.mp4',
 'reel_PSYCHOLOGICAL_INSIGHT.mp4',
 'reel_CONCEPT.mp4']

# FFMPG my code

In [ ]:
import ffmpeg
import random
import os
import glob
import shutil

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/"
OUTPUT_DIR_BASE = "output_reels_v3"



TRANSITION_DURATION = 0.5   
REEL_WIDTH = 720           
REEL_HEIGHT = 1280          
OUTPUT_FPS = 30  


VIDEO_CODEC = 'libx264'     
AUDIO_CODEC = 'aac'         
AUDIO_BITRATE = '128k'      
CRF = 23                    
PRESET_X264 = 'medium'

XFADE_TRANSITIONS = ['fade']
CONCAT_REEL_TRANSITION_TYPE = 'fade'
CONCAT_REEL_TRANSITION_DURATION = 0.7

def get_duration(filename):
    """Gets the duration of a media file."""
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except ffmpeg.Error as e:
        print(f"FFmpeg error probing duration for {filename}: {e.stderr.decode('utf8') if e.stderr else str(e)}")
        return None
    except Exception:
        print(f"Error probing duration for {filename}: {str(e)}")
        return None

get_duration(os.path.join(BASE_VIDEOS_PATH, 'my_test_files/videos/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/visuals'))

In [17]:
os.listdir('/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back/visuals/section_CONCEPT')

['CONCEPT_6100413.mp4', 'CONCEPT_6623022.mp4', 'CONCEPT_6068298.mp4']

In [14]:
media_path = '/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/audio'

for i in os.listdir(media_path):
  if i.endswith(('.mp4', '.wav')):
    vid_dur = get_duration(os.path.join(media_path, i))
    print(i, " ", vid_dur)

REAL-WORLD_EXAMPLE.wav   29.568
CTA.wav   6.229333
CONCEPT.wav   11.669333
PSYCHOLOGICAL_INSIGHT.wav   12.394667
ACTIONABLE_TIP.wav   11.306667
HOOK.wav   4.266667


In [ ]:
media_path = '/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/visuals'

for i in os.listdir(media_path):
  if i.endswith('.mp4'):
    vid_dur = get_duration(os.path.join(media_path, i))
    print(i, " ", vid_dur)

In [ ]:
import ffmpeg
import random
import os

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/"
OUTPUT_DIR_BASE = "output_reels_fades_only"

MIN_SEGMENT_DURATION = 2.0
MAX_SEGMENT_DURATION = 5.0
TRANSITION_DURATION = 0.5

REEL_WIDTH = 720
REEL_HEIGHT = 1280
OUTPUT_FPS = 30

VIDEO_CODEC = 'libx264'
AUDIO_CODEC = 'aac'
AUDIO_BITRATE = '128k'
CRF = 23
PRESET_X264 = 'medium'

XFADE_TRANSITIONS = ['fade']


def get_duration(filename):
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except Exception as e:
        print(f"Error probing duration for {filename}: {e}")
        return None


def apply_segment_effects(video_node):
    node = video_node.filter('scale',
                              w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                              h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)', eval='frame')
    node = node.filter('crop', w=REEL_WIDTH, h=REEL_HEIGHT,
                       x=f'(iw-{REEL_WIDTH})/2', y=f'(ih-{REEL_HEIGHT})/2')
    node = node.filter('fps', fps=OUTPUT_FPS, round='near')
    node = node.filter('setpts', 'PTS-STARTPTS')
    return node


def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None or not associated_video_files:
        print(f"Skipping {audio_file_path}, missing audio/video.")
        return

    print(f"\nProcessing {os.path.basename(audio_file_path)}, target {audio_duration:.2f}s")
    segments = []
    total = 0
    idx = 0
    files = associated_video_files.copy()
    random.shuffle(files)

    while total < audio_duration and files:
        path = files[idx % len(files)]
        dur = get_duration(path) or 0
        if dur < 0.5:
            idx += 1; continue
        remain = audio_duration - total
        seg_dur = min(max(MIN_SEGMENT_DURATION, random.uniform(MIN_SEGMENT_DURATION, MAX_SEGMENT_DURATION)), remain, dur)
        start = random.uniform(0, max(0, dur - seg_dur))
        print(f"  Segment {os.path.basename(path)} ss={start:.2f} t={seg_dur:.2f}")
        inp = ffmpeg.input(path, ss=start, t=seg_dur)
        vid = apply_segment_effects(inp.video)
        segments.append({'stream': vid, 'duration': seg_dur})
        total += seg_dur; idx += 1
        if total >= audio_duration:
            overs = total - audio_duration
            if overs > 0:
                segments[-1]['duration'] = max(0.1, segments[-1]['duration'] - overs)
                total = sum(s['duration'] for s in segments)
            break

    if not segments:
        print(f"No segments for {audio_file_path}")
        return

    # merge with xfade
    merged = segments[0]['stream']
    timeline = segments[0]['duration']
    for seg in segments[1:]:
        trans = XFADE_TRANSITIONS[0]
        off = max(0, timeline - TRANSITION_DURATION)
        print(f"    xfade dur={TRANSITION_DURATION} off={off:.2f}")
        merged = ffmpeg.filter([merged, seg['stream']], 'xfade', transition=trans,
                               duration=TRANSITION_DURATION, offset=off)
        merged = merged.filter('setpts', 'PTS-STARTPTS')
        timeline += seg['duration'] - TRANSITION_DURATION

    audio = ffmpeg.input(audio_file_path).audio
    try:
        out = ffmpeg.output(merged, audio, output_file_path,
                             vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
                             audio_bitrate=AUDIO_BITRATE,
                             crf=CRF, preset=PRESET_X264,
                             movflags='+faststart', shortest=None)
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        print(f"  Writing reel to {output_file_path}")
        ffmpeg.run(out, overwrite_output=True)
    except Exception as e:
        print(f"Error writing {output_file_path}: {e}")


def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    scripts = [d for d in os.listdir(BASE_VIDEOS_PATH)
               if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script in scripts:
        base = os.path.join(BASE_VIDEOS_PATH, script)
        audio_dir = os.path.join(base, 'audio')
        vis_root = os.path.join(base, 'visuals')
        out_root = os.path.join(OUTPUT_DIR_BASE, script)
        os.makedirs(out_root, exist_ok=True)
        if not os.path.isdir(audio_dir) or not os.path.isdir(vis_root): continue

        print(f"\nScript: {script}")
        for audio_file in os.listdir(audio_dir):
            if not audio_file.lower().endswith(('.wav','.mp3','.m4a','.aac')): continue
            name = os.path.splitext(audio_file)[0]
            section_dir = os.path.join(vis_root, f'section_{name}')
            if os.path.isdir(section_dir):
                vids = [os.path.join(section_dir, f)
                        for f in os.listdir(section_dir)
                        if f.lower().endswith('.mp4')]
            else:
                vids = [os.path.join(vis_root, f)
                        for f in os.listdir(vis_root)
                        if f.lower().endswith(('.mp4','.mov','.avi','.mkv'))]
            audio_path = os.path.join(audio_dir, audio_file)
            out_path = os.path.join(out_root, f'reel_{name}.mp4')
            create_reel_for_audio(audio_path, vids, out_path)

if __name__ == '__main__':
    main()


In [ ]:
import ffmpeg
import random
import os

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/"
OUTPUT_DIR_BASE = "output_reels_fades_only"

MIN_SEGMENT_DURATION = 2.0
MAX_SEGMENT_DURATION = 5.0
TRANSITION_DURATION = 0.5
SILENCE_DURATION = 1.5  # breathing room after each section

REEL_WIDTH = 720
REEL_HEIGHT = 1280
OUTPUT_FPS = 30

VIDEO_CODEC = 'libx264'
AUDIO_CODEC = 'aac'
AUDIO_BITRATE = '128k'
CRF = 23
PRESET_X264 = 'medium'

XFADE_TRANSITIONS = ['fade']


def get_duration(filename):
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except Exception as e:
        print(f"Error probing duration for {filename}: {e}")
        return None


def apply_segment_effects(video_node):
    node = video_node.filter('scale',
                              w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
                              h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)', eval='frame')
    node = node.filter('crop', w=REEL_WIDTH, h=REEL_HEIGHT,
                       x=f'(iw-{REEL_WIDTH})/2', y=f'(ih-{REEL_HEIGHT})/2')
    node = node.filter('fps', fps=OUTPUT_FPS, round='near')
    node = node.filter('setpts', 'PTS-STARTPTS')
    return node


def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    # 1. Measure durations
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None or not associated_video_files:
        print(f"Skipping {audio_file_path}, missing audio/video.")
        return
    target_duration = audio_duration + SILENCE_DURATION
    print(f"\nProcessing {os.path.basename(audio_file_path)}: audio {audio_duration:.2f}s + silence {SILENCE_DURATION:.2f}s = target {target_duration:.2f}s")

    # 2. Select & trim clips to fill target_duration
    segments = []
    total = 0.0
    idx = 0
    files = associated_video_files.copy()
    random.shuffle(files)

    while total < target_duration and files:
        path = files[idx % len(files)]
        dur = get_duration(path) or 0.0
        if dur < 0.5:
            idx += 1
            continue
        # Determine segment length
        remaining = target_duration - total
        seg_len = min(
            remaining,
            dur,
            random.uniform(MIN_SEGMENT_DURATION, MAX_SEGMENT_DURATION)
        )
        # Ensure minimum if enough remains
        if remaining > MIN_SEGMENT_DURATION and seg_len < MIN_SEGMENT_DURATION:
            seg_len = min(remaining, dur, MIN_SEGMENT_DURATION)
        # Compute random start
        start = random.uniform(0, max(0, dur - seg_len))
        print(f"  Segment {os.path.basename(path)} ss={start:.2f} t={seg_len:.2f}")
        inp = ffmpeg.input(path, ss=start, t=seg_len)
        vid = apply_segment_effects(inp.video)
        segments.append({'stream': vid, 'duration': seg_len})
        total += seg_len
        idx += 1
        # Trim overshoot
        if total >= target_duration:
            overshoot = total - target_duration
            if overshoot > 0:
                segments[-1]['duration'] = max(0.1, segments[-1]['duration'] - overshoot)
                total = sum(s['duration'] for s in segments)
            break

    if not segments:
        print(f"No segments for {audio_file_path}")
        return

    print(f"  Total video length: {total:.2f}s, matches target {target_duration:.2f}s")

    # 3. Concatenate with crossfades
    merged = segments[0]['stream']
    timeline = segments[0]['duration']
    for seg in segments[1:]:
        trans = XFADE_TRANSITIONS[0]
        offset = max(0, timeline - TRANSITION_DURATION)
        print(f"    xfade dur={TRANSITION_DURATION:.2f}s at off={offset:.2f}s")
        merged = ffmpeg.filter(
            [merged, seg['stream']], 'xfade',
            transition=trans,
            duration=TRANSITION_DURATION,
            offset=offset
        ).filter('setpts', 'PTS-STARTPTS')
        timeline += seg['duration'] - TRANSITION_DURATION

    # 4. Build padded audio (TTS + silence)
    tts = ffmpeg.input(audio_file_path)
    silence = ffmpeg.input(
        'anullsrc=channel_layout=stereo:sample_rate=44100',
        format='lavfi', t=SILENCE_DURATION
    )
    padded_audio = ffmpeg.filter([tts.audio, silence.audio], 'concat', n=2, v=0, a=1)

    # 5. Merge video + padded audio
    try:
        out = ffmpeg.output(
            merged, padded_audio,
            output_file_path,
            vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
            audio_bitrate=AUDIO_BITRATE,
            crf=CRF, preset=PRESET_X264,
            movflags='+faststart', shortest=None
        )
        os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
        print(f"  Writing reel: {output_file_path}")
        ffmpeg.run(out, overwrite_output=True)
        print(f"Completed: {output_file_path}")
    except Exception as e:
        print(f"Error writing {output_file_path}: {e}")


def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    scripts = [d for d in os.listdir(BASE_VIDEOS_PATH)
               if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script in scripts:
        base = os.path.join(BASE_VIDEOS_PATH, script)
        audio_dir = os.path.join(base, 'audio')
        visuals_root = os.path.join(base, 'visuals')
        out_root = os.path.join(OUTPUT_DIR_BASE, script)
        os.makedirs(out_root, exist_ok=True)
        if not os.path.isdir(audio_dir) or not os.path.isdir(visuals_root):
            continue

        print(f"\nScript folder: {script}")
        for audio_file in os.listdir(audio_dir):
            if not audio_file.lower().endswith(('.wav', '.mp3', '.m4a', '.aac')):
                continue
            name = os.path.splitext(audio_file)[0]
            # Prefer section_name folder
            section_folder = os.path.join(visuals_root, f'section_{name}')
            if os.path.isdir(section_folder):
                vids = [os.path.join(section_folder, f)
                        for f in os.listdir(section_folder)
                        if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]
            else:
                vids = [os.path.join(visuals_root, f)
                        for f in os.listdir(visuals_root)
                        if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]

            audio_path = os.path.join(audio_dir, audio_file)
            out_path = os.path.join(out_root, f'reel_{name}.mp4')
            create_reel_for_audio(audio_path, vids, out_path)

if __name__ == '__main__':
    main()



Script folder: Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back

Processing REAL-WORLD_EXAMPLE.wav: audio 33.32s + silence 1.50s = target 34.82s
  Segment REAL-WORLD_EXAMPLE_6930970.mp4 ss=1.95 t=4.49
  Segment REAL-WORLD_EXAMPLE_6565540.mp4 ss=16.99 t=2.72
  Segment REAL-WORLD_EXAMPLE_8873041.mp4 ss=1.98 t=2.59
  Segment REAL-WORLD_EXAMPLE_6930970.mp4 ss=5.59 t=4.78
  Segment REAL-WORLD_EXAMPLE_6565540.mp4 ss=5.08 t=4.90
  Segment REAL-WORLD_EXAMPLE_8873041.mp4 ss=3.32 t=3.74
  Segment REAL-WORLD_EXAMPLE_6930970.mp4 ss=5.58 t=4.14
  Segment REAL-WORLD_EXAMPLE_6565540.mp4 ss=7.65 t=3.00
  Segment REAL-WORLD_EXAMPLE_8873041.mp4 ss=5.27 t=4.46
  Total video length: 34.82s, matches target 34.82s
    xfade dur=0.50s at off=3.99s
    xfade dur=0.50s at off=6.21s
    xfade dur=0.50s at off=8.30s
    xfade dur=0.50s at off=12.58s
    xfade dur=0.50s at off=16.98s
    xfade dur=0.50s at off=20.22s
    xfade dur=0.50s at off=23.86s
    xfade dur=0.50s at off=26.36s
  Writing reel: output_

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back/reel_REAL-WORLD_EXAMPLE.mp4

Processing CTA.wav: audio 5.61s + silence 1.50s = target 7.11s
  Segment CTA_5552584.mp4 ss=0.61 t=4.71
  Segment CTA_7224512.mp4 ss=3.64 t=2.40
  Total video length: 7.11s, matches target 7.11s
    xfade dur=0.50s at off=4.21s
  Writing reel: output_reels_fades_only/Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back/reel_CTA.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back/reel_CTA.mp4

Processing CONCEPT.wav: audio 11.82s + silence 1.50s = target 13.32s
  Segment CONCEPT_6100413.mp4 ss=11.90 t=4.75
  Segment CONCEPT_6068298.mp4 ss=7.32 t=4.28
  Segment CONCEPT_6623022.mp4 ss=1.02 t=2.90
  Segment CONCEPT_6100413.mp4 ss=13.48 t=1.38
  Total video length: 13.32s, matches target 13.32s
    xfade dur=0.50s at off=4.25s
    xfade dur=0.50s at off=8.04s
    xfade dur=0.50s at off=10.44s
  Writing reel: output_reels_fades_only/Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back/reel_CONCEPT.mp4


...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/Why_Your_Best_Ideas_Get_RejectedAnd_How_to_Fight_Back/reel_HOOK.mp4

Script folder: The_Semmelweis_Reflex_Why_We_Reject_the_Truth_That_Could_Save_Us
Skipping /home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Semmelweis_Reflex_Why_We_Reject_the_Truth_That_Could_Save_Us/audio/REAL-WORLD_EXAMPLE.wav, missing audio/video.
Skipping /home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Semmelweis_Reflex_Why_We_Reject_the_Truth_That_Could_Save_Us/audio/CTA.wav, missing audio/video.
Skipping /home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Semmelweis_Reflex_Why_We_Reject_the_Truth_That_Could_Save_Us/audio/CONCEPT.wav, missing audio/video.
Skipping /home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/The_Semmelweis_Reflex_Why_We_Reject_the_Truth_That_Could_Save_Us/audio/PSYCHOLOGICAL_INSIGHT.wav, missing audio/video.
Skipping /home/aditya-ladawa/Aditya/z_proje

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/The_Semmelweis_Reflex_Why_We_Reject_the_Truth_That_Could_Save_Us/reel_HOOK.mp4


[mp4 @ 0x6517374fc280] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x6517374fc140] video:2510kB audio:175kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.432644%
frame=  397 fps= 56 q=-1.0 Lsize=    2697kB time=00:00:13.18 bitrate=1675.7kbits/s speed=1.84x    
[libx264 @ 0x6517374fd2c0] frame I:2     Avg QP:20.22  size: 28888
[libx264 @ 0x6517374fd2c0] frame P:111   Avg QP:22.94  size: 14570
[libx264 @ 0x6517374fd2c0] frame B:284   Avg QP:25.56  size:  3150
[libx264 @ 0x6517374fd2c0] consecutive B-frames:  1.0%  7.1% 11.3% 80.6%
[libx264 @ 0x6517374fd2c0] mb I  I16..4: 24.6% 64.5% 10.9%
[libx264 @ 0x6517374fd2c0] mb P  I16..4:  7.6% 18.5%  1.0%  P16..4: 33.1%  7.9%  3.2%  0.0%  0.0%    skip:28.7%
[libx264 @ 0x6517374fd2c0] mb B  I16..4:  0.5%  1.0%  0.0%  B16..8: 29.2%  1.4%  0.2%  direct: 1.4%  skip:66.2%  L0:43.5% L1:49.0% BI: 7.4%
[libx264 @ 0x6517374fd2c0] 8x8 transform intra:67.5% inter:91.2%
[libx264 @ 0x6517374fd2c0

In [1]:
import ffmpeg
import random
import os

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/"
OUTPUT_DIR_BASE = "output_reels_fades_only"

MIN_SEGMENT_DURATION = 2.0
MAX_SEGMENT_DURATION = 5.0
TRANSITION_DURATION = 0.5
REEL_WIDTH = 720
REEL_HEIGHT = 1280
OUTPUT_FPS = 30

VIDEO_CODEC = 'libx264'
AUDIO_CODEC = 'aac'
AUDIO_BITRATE = '128k'
CRF = 23
PRESET_X264 = 'medium'
XFADE_TRANSITIONS = ['fade']

# Additional silence per section
SECTION_SILENCE = {
    'HOOK': 1.0,
    'CONCEPT': 1.3,
    'REAL-WORLD_EXAMPLE': 1.5,
    'PSYCHOLOGICAL_INSIGHT': 2.0,
    'ACTIONABLE_TIP': 1.2,
    'CTA': 0.8
}


def get_duration(filename):
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except Exception as e:
        print(f"Error probing duration for {filename}: {e}")
        return None


def apply_segment_effects(video_node):
    node = video_node.filter(
        'scale', w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
        h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)', eval='frame'
    )
    node = node.filter('crop', w=REEL_WIDTH, h=REEL_HEIGHT,
                       x=f'(iw-{REEL_WIDTH})/2', y=f'(ih-{REEL_HEIGHT})/2')
    node = node.filter('fps', fps=OUTPUT_FPS, round='near')
    node = node.filter('setpts', 'PTS-STARTPTS')
    return node


def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    section_name = os.path.splitext(os.path.basename(audio_file_path))[0]
    section_key = section_name.upper()
    silence_duration = SECTION_SILENCE.get(section_key, 1.0)

    # 1. Measure audio
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None or not associated_video_files:
        print(f"Skipping {audio_file_path}, missing audio/video.")
        return None

    target_duration = audio_duration + silence_duration
    print(f"\nProcessing {section_name}: audio {audio_duration:.2f}s + silence {silence_duration:.2f}s = target {target_duration:.2f}s")

    # 2. Collect segments until filling target_duration
    segments = []
    total = 0.0
    files = associated_video_files.copy()
    random.shuffle(files)
    idx = 0

    while total < target_duration:
        path = files[idx % len(files)]
        idx += 1
        dur = get_duration(path) or 0.0
        if dur < 0.5:
            continue

        remaining = target_duration - total
        seg_len = min(remaining, dur, random.uniform(MIN_SEGMENT_DURATION, MAX_SEGMENT_DURATION))
        if remaining > MIN_SEGMENT_DURATION and seg_len < MIN_SEGMENT_DURATION:
            seg_len = min(remaining, dur, MIN_SEGMENT_DURATION)

        start = random.uniform(0, max(0, dur - seg_len))
        print(f"  Segment {os.path.basename(path)} ss={start:.2f} t={seg_len:.2f}")
        inp = ffmpeg.input(path, ss=start, t=seg_len)
        vid = apply_segment_effects(inp.video)
        segments.append({'stream': vid, 'duration': seg_len})
        total += seg_len

    print(f"  Total video length: {total:.2f}s (target {target_duration:.2f}s)")

    # 3. Concatenate with crossfades
    merged = segments[0]['stream']
    timeline = segments[0]['duration']
    for seg in segments[1:]:
        offset = max(0, timeline - TRANSITION_DURATION)
        print(f"    xfade dur={TRANSITION_DURATION:.2f}s at off={offset:.2f}s")
        merged = ffmpeg.filter(
            [merged, seg['stream']], 'xfade',
            transition=XFADE_TRANSITIONS[0],
            duration=TRANSITION_DURATION,
            offset=offset
        ).filter('setpts', 'PTS-STARTPTS')
        timeline += seg['duration'] - TRANSITION_DURATION

    # 4. Build padded audio
    tts = ffmpeg.input(audio_file_path)
    silence = ffmpeg.input(
        'anullsrc=channel_layout=stereo:sample_rate=44100',
        format='lavfi', t=silence_duration
    )
    padded_audio = ffmpeg.filter([tts.audio, silence.audio], 'concat', n=2, v=0, a=1)

    # 5. Merge & output
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    try:
        out = ffmpeg.output(
            merged, padded_audio,
            output_file_path,
            vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
            audio_bitrate=AUDIO_BITRATE,
            crf=CRF, preset=PRESET_X264,
            movflags='+faststart'
        )
        print(f"  Writing reel: {output_file_path}")
        ffmpeg.run(out, overwrite_output=True)
        print(f"Completed: {output_file_path}")
        return output_file_path
    except Exception as e:
        print(f"Error writing {output_file_path}: {e}")
        return None


def concatenate_sections(section_files, final_output):
    list_path = os.path.join(os.path.dirname(final_output), 'sections_to_concat.txt')
    with open(list_path, 'w') as f:
        for filepath in section_files:
            f.write(f"file '{os.path.abspath(filepath)}'\n")

    print(f"Concatenating {len(section_files)} sections into {final_output}")
    try:
        ffmpeg.input(list_path, format='concat', safe=0).output(
            final_output,
            vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
            audio_bitrate=AUDIO_BITRATE,
            crf=CRF, preset=PRESET_X264,
            movflags='+faststart'
        ).run(overwrite_output=True)
        print(f"Final reel created: {final_output}")
    except Exception as e:
        print(f"Error concatenating final reel: {e}")


def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    scripts = [d for d in os.listdir(BASE_VIDEOS_PATH)
               if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script in scripts:
        base = os.path.join(BASE_VIDEOS_PATH, script)
        audio_dir = os.path.join(base, 'audio')
        visuals_root = os.path.join(base, 'visuals')
        out_root = os.path.join(OUTPUT_DIR_BASE, script)
        os.makedirs(out_root, exist_ok=True)
        if not os.path.isdir(audio_dir) or not os.path.isdir(visuals_root):
            continue

        print(f"\nScript folder: {script}")
        section_outputs = []
        for audio_file in os.listdir(audio_dir):
            if not audio_file.lower().endswith(('.wav', '.mp3', '.m4a', '.aac')):
                continue
            vids = []
            section_name = os.path.splitext(audio_file)[0]
            section_folder = os.path.join(visuals_root, f'section_{section_name}')
            if os.path.isdir(section_folder):
                vids = [os.path.join(section_folder, f)
                        for f in os.listdir(section_folder)
                        if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]
            else:
                vids = [os.path.join(visuals_root, f)
                        for f in os.listdir(visuals_root)
                        if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv'))]

            audio_path = os.path.join(audio_dir, audio_file)
            out_path = os.path.join(out_root, f'reel_{section_name}.mp4')
            result = create_reel_for_audio(audio_path, vids, out_path)
            if result:
                section_outputs.append(result)

        if section_outputs:
            final_out = os.path.join(out_root, f'{script}_final_reel.mp4')
            concatenate_sections(section_outputs, final_out)

if __name__ == '__main__':
    main()



Script folder: Why_You_Underestimate_RisksAnd_How_It_Costs_You

Processing REAL-WORLD_EXAMPLE: audio 33.47s + silence 1.50s = target 34.97s
  Segment REAL-WORLD_EXAMPLE_4961923.mp4 ss=1.16 t=3.56
  Segment REAL-WORLD_EXAMPLE_6580032.mp4 ss=2.72 t=3.91
  Segment REAL-WORLD_EXAMPLE_8170425.mp4 ss=2.08 t=2.97
  Segment REAL-WORLD_EXAMPLE_5971265.mp4 ss=12.99 t=2.63
  Segment REAL-WORLD_EXAMPLE_6930644.mp4 ss=2.72 t=3.87
  Segment REAL-WORLD_EXAMPLE_7247827.mp4 ss=3.89 t=2.76
  Segment REAL-WORLD_EXAMPLE_4961923.mp4 ss=6.69 t=4.34
  Segment REAL-WORLD_EXAMPLE_6580032.mp4 ss=26.83 t=2.55
  Segment REAL-WORLD_EXAMPLE_8170425.mp4 ss=4.68 t=2.90
  Segment REAL-WORLD_EXAMPLE_5971265.mp4 ss=13.15 t=3.97
  Segment REAL-WORLD_EXAMPLE_6930644.mp4 ss=16.94 t=1.52
  Total video length: 34.97s (target 34.97s)
    xfade dur=0.50s at off=3.06s
    xfade dur=0.50s at off=6.47s
    xfade dur=0.50s at off=8.94s
    xfade dur=0.50s at off=11.06s
    xfade dur=0.50s at off=14.43s
    xfade dur=0.50s at off=

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_REAL-WORLD_EXAMPLE.mp4

Processing CTA: audio 5.38s + silence 0.80s = target 6.18s
  Segment CTA_9943144.mp4 ss=2.91 t=4.16
  Segment CTA_15019286.mp4 ss=7.71 t=2.01
  Total video length: 6.18s (target 6.18s)
    xfade dur=0.50s at off=3.66s
  Writing reel: output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_CTA.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_CTA.mp4

Processing CONCEPT: audio 15.25s + silence 1.30s = target 16.55s
  Segment CONCEPT_8544033.mp4 ss=5.98 t=4.41
  Segment CONCEPT_8544030.mp4 ss=4.41 t=2.81
  Segment CONCEPT_8543925.mp4 ss=8.18 t=2.17
  Segment CONCEPT_8544033.mp4 ss=0.03 t=3.55
  Segment CONCEPT_8544030.mp4 ss=1.51 t=3.61
  Total video length: 16.55s (target 16.55s)
    xfade dur=0.50s at off=3.91s
    xfade dur=0.50s at off=6.22s
    xfade dur=0.50s at off=7.89s
    xfade dur=0.50s at off=10.94s
  Writing reel: output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_CONCEPT.mp4


...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_HOOK.mp4
Concatenating 6 sections into output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/Why_You_Underestimate_RisksAnd_How_It_Costs_You_final_reel.mp4


[libx264 @ 0x6096c2c4b680] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x6096c2c4b680] profile High 4:4:4 Predictive, level 3.1, 4:4:4, 8-bit
[libx264 @ 0x6096c2c4b680] 264 - core 164 r3108 31e19f9 - H.264/MPEG-4 AVC codec - Copyleft 2003-2023 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=4 threads=12 lookahead_threads=2 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to 'output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/Why_You_Underestimate_RisksAnd_How_It_C

Final reel created: output_reels_fades_only/Why_You_Underestimate_RisksAnd_How_It_Costs_You/Why_You_Underestimate_RisksAnd_How_It_Costs_You_final_reel.mp4


[mp4 @ 0x6096c2ceea40] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x6096c2c47140] video:12359kB audio:1373kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.634325%
frame= 3313 fps= 79 q=-1.0 Lsize=   13819kB time=00:01:52.42 bitrate=1007.0kbits/s dup=392 drop=0 speed=2.66x    
[libx264 @ 0x6096c2c4b680] frame I:20    Avg QP:18.27  size: 28404
[libx264 @ 0x6096c2c4b680] frame P:991   Avg QP:20.28  size:  8864
[libx264 @ 0x6096c2c4b680] frame B:2302  Avg QP:23.96  size:  1435
[libx264 @ 0x6096c2c4b680] consecutive B-frames:  1.6% 15.3%  6.2% 76.9%
[libx264 @ 0x6096c2c4b680] mb I  I16..4: 21.2% 72.1%  6.7%
[libx264 @ 0x6096c2c4b680] mb P  I16..4:  5.1% 11.6%  0.6%  P16..4: 30.5%  6.7%  2.5%  0.0%  0.0%    skip:43.1%
[libx264 @ 0x6096c2c4b680] mb B  I16..4:  0.2%  0.4%  0.0%  B16..8: 22.2%  0.7%  0.1%  direct: 0.3%  skip:76.1%  L0:47.2% L1:48.8% BI: 4.0%
[libx264 @ 0x6096c2c4b680] 8x8 transform intra:67.2% inter:90.9%
[libx264

In [ ]:
import ffmpeg
import random
import os

# --- Configuration --- ( zůstává stejné )
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/" # Upravte podle potřeby
OUTPUT_DIR_BASE = "output_reels_fades_only_v2" # Můžete změnit výstupní složku

MIN_SEGMENT_DURATION = 2.0
MAX_SEGMENT_DURATION = 5.0
TRANSITION_DURATION = 0.5
REEL_WIDTH = 720
REEL_HEIGHT = 1280
OUTPUT_FPS = 30

VIDEO_CODEC = 'libx264'
AUDIO_CODEC = 'aac'
AUDIO_BITRATE = '128k'
CRF = 23
PRESET_X264 = 'medium'
XFADE_TRANSITIONS = ['fade']

SECTION_SILENCE = {
    'HOOK': 1.0,
    'CONCEPT': 1.3,
    'REAL-WORLD_EXAMPLE': 1.5,
    'PSYCHOLOGICAL_INSIGHT': 2.0,
    'ACTIONABLE_TIP': 1.2,
    'CTA': 0.8
}


def get_duration(filename):
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except Exception as e:
        print(f"Error probing duration for {filename}: {e}")
        return None


def apply_segment_effects(video_node):
    node = video_node.filter(
        'scale', w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
        h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)', eval='frame'
    )
    node = node.filter('crop', w=REEL_WIDTH, h=REEL_HEIGHT,
                       x=f'(iw-{REEL_WIDTH})/2', y=f'(ih-{REEL_HEIGHT})/2')
    node = node.filter('fps', fps=OUTPUT_FPS, round='near')
    node = node.filter('setpts', 'PTS-STARTPTS')
    return node


def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    section_name = os.path.splitext(os.path.basename(audio_file_path))[0]
    section_key = section_name.upper()
    silence_duration = SECTION_SILENCE.get(section_key, 1.0)

    # 1. Measure audio
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None:
        print(f"Skipping {audio_file_path}, could not get audio duration.")
        return None
    if not associated_video_files:
        print(f"Skipping {audio_file_path}, no associated video files.")
        return None

    target_duration = audio_duration + silence_duration
    print(f"\nProcessing {section_name}: audio {audio_duration:.2f}s + silence {silence_duration:.2f}s = target {target_duration:.2f}s")

    # 2. Collect segments until filling target_duration (NEW LOGIC)
    segments = []  # List of {'stream': video_node, 'raw_duration': float}
    effective_video_duration = 0.0
    
    files_to_use = associated_video_files.copy()
    random.shuffle(files_to_use)
    
    # Max attempts to prevent infinite loops if videos are unsuitable
    max_segment_picking_attempts = len(files_to_use) * 3 + 5 # Allow cycling and some grace
    current_attempt_idx = 0
    
    # Index for cycling through files_to_use
    file_picker_idx = 0

    while effective_video_duration < target_duration and current_attempt_idx < max_segment_picking_attempts:
        current_attempt_idx += 1

        if not files_to_use: # Should have been caught earlier, but as a safeguard
            print("  Error: No video files available to pick from.")
            break

        video_path = files_to_use[file_picker_idx % len(files_to_use)]
        source_video_duration = get_duration(video_path) or 0.0

        needed_effective_contribution = target_duration - effective_video_duration
        if needed_effective_contribution <= 0.01 : # Target effectively met
             break

        segment_raw_duration = 0.0
        is_first_segment = not bool(segments)

        min_raw_len_this_segment = MIN_SEGMENT_DURATION
        max_raw_len_this_segment = MAX_SEGMENT_DURATION

        if is_first_segment:
            # For the first segment, raw_duration is its effective contribution.
            # We want to pick a segment of length that is ideally needed_effective_contribution,
            # constrained by MIN_SEGMENT_DURATION, MAX_SEGMENT_DURATION, and source_video_duration.
            
            # Ideal length is what's needed, capped by MAX_SEGMENT_DURATION.
            ideal_len = min(needed_effective_contribution, max_raw_len_this_segment)
            # It must be at least MIN_SEGMENT_DURATION, unless ideal_len itself is smaller (for very short targets).
            segment_raw_duration = max(ideal_len, min_raw_len_this_segment if ideal_len >= min_raw_len_this_segment else ideal_len)
            # And finally, capped by what the source video actually has.
            segment_raw_duration = min(segment_raw_duration, source_video_duration)

        else: # Subsequent segment
            min_raw_len_this_segment = max(MIN_SEGMENT_DURATION, TRANSITION_DURATION + 0.01) # Must be longer than transition
            
            # Target raw length for this segment to fulfill the needed effective contribution:
            target_raw_len_for_segment = needed_effective_contribution + TRANSITION_DURATION
            
            # Ideal length is this target_raw_len, capped by MAX_SEGMENT_DURATION
            ideal_len = min(target_raw_len_for_segment, max_raw_len_this_segment)
            # It must be at least min_raw_len_this_segment, unless ideal_len itself is smaller.
            segment_raw_duration = max(ideal_len, min_raw_len_this_segment if ideal_len >= min_raw_len_this_segment else ideal_len)
            # And finally, capped by what the source video actually has.
            segment_raw_duration = min(segment_raw_duration, source_video_duration)

        # Validate the calculated segment_raw_duration
        valid_segment = True
        if segment_raw_duration < 0.1: # Too short
            valid_segment = False
        if not is_first_segment and segment_raw_duration <= TRANSITION_DURATION: # Subsequent segment not long enough for transition
            valid_segment = False
        
        if not valid_segment:
            print(f"  Skipping {os.path.basename(video_path)}: calculated raw_t ({segment_raw_duration:.2f}s) is unsuitable. Source_t: {source_video_duration:.2f}s.")
            file_picker_idx += 1 # Try next file in the list
            continue # Next attempt in the while loop

        # If we are here, segment_raw_duration is good.
        start_time = random.uniform(0, max(0, source_video_duration - segment_raw_duration))
        print(f"  Segment from {os.path.basename(video_path)}: ss={start_time:.2f}, raw_t={segment_raw_duration:.2f}s")
        
        inp = ffmpeg.input(video_path, ss=start_time, t=segment_raw_duration)
        vid_node = apply_segment_effects(inp.video)
        segments.append({'stream': vid_node, 'raw_duration': segment_raw_duration})

        if is_first_segment:
            effective_video_duration += segment_raw_duration
        else:
            effective_video_duration += segment_raw_duration - TRANSITION_DURATION
        
        file_picker_idx += 1 # Move to next file choice for variety in next segment

    if not segments:
        print(f"Error: No video segments could be collected for {audio_file_path}. Target: {target_duration:.2f}s, Effective Found: {effective_video_duration:.2f}s")
        return None

    print(f"  Collected {len(segments)} segments. Total effective video length: {effective_video_duration:.2f}s (target {target_duration:.2f}s)")
    
    # If effective_video_duration is slightly less than target_duration due to segment constraints,
    # the last frame of video might still hold for a very short period.
    # If it's slightly more, ffmpeg might trim the video or audio depending on other flags (not set here to do so explicitly).
    # Ideally, effective_video_duration will be very close to target_duration.

    # 3. Concatenate with crossfades
    merged_video_stream = segments[0]['stream']
    # 'timeline' will track the duration of the 'merged_video_stream'
    timeline = segments[0]['raw_duration'] 

    for i in range(1, len(segments)):
        next_segment_data = segments[i]
        # The offset for xfade is the duration of the video built so far, MINUS the transition duration
        xfade_offset = max(0, timeline - TRANSITION_DURATION) 
        
        print(f"    Applying xfade: transition={XFADE_TRANSITIONS[0]}, duration={TRANSITION_DURATION:.2f}s, offset={xfade_offset:.2f}s")
        
        merged_video_stream = ffmpeg.filter(
            [merged_video_stream, next_segment_data['stream']], 'xfade',
            transition=XFADE_TRANSITIONS[0], # Using the first (and only) transition type
            duration=TRANSITION_DURATION,
            offset=xfade_offset
        ).filter('setpts', 'PTS-STARTPTS') # Reset PTS for the output of xfade operation

        # Update the total duration of the 'merged_video_stream'
        timeline += next_segment_data['raw_duration'] - TRANSITION_DURATION
    
    print(f"  Final calculated video duration after xfades should be: {timeline:.2f}s")

    # 4. Build padded audio (Remains the same)
    tts_input = ffmpeg.input(audio_file_path)
    # Ensure anullsrc uses a sample rate compatible with common audio, e.g., 44100 or 48000
    # Matching the TTS audio's sample rate is best if known, otherwise a common one.
    silence_input = ffmpeg.input(
        'anullsrc=channel_layout=stereo:sample_rate=44100', # Or use 48000
        format='lavfi', t=silence_duration
    )
    padded_audio = ffmpeg.filter([tts_input.audio, silence_input.audio], 'concat', n=2, v=0, a=1)

    # 5. Merge & output (Remains mostly the same)
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    try:
        # The final output video will have the duration of the longest stream (video or audio).
        # Since `padded_audio` is exactly `target_duration`, and we tried to make `merged_video_stream`
        # (whose duration is `timeline`) also `target_duration`, they should match closely.
        # If `timeline` is slightly shorter, the last video frame will hold.
        # If `timeline` is slightly longer, the audio might end first or video gets trimmed if `-shortest` was used.
        # We want video to match audio, so it's implicitly handled by ffmpeg by padding the shorter one.
        
        out_node = ffmpeg.output(
            merged_video_stream, padded_audio,
            output_file_path,
            vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
            audio_bitrate=AUDIO_BITRATE,
            crf=CRF, preset=PRESET_X264,
            # To ensure the output duration is primarily dictated by the audio (which has the precise target_duration)
            # especially if video ends up slightly longer than audio due to segment choices,
            # you could add shortest, but this might cut planned video.
            # The current setup will make the output last as long as the longest input.
            # If effective_video_duration was carefully managed to be target_duration, this is fine.
            # movflags='+faststart' is good.
            movflags='+faststart',
            # Force video duration to match audio duration if there's a small discrepancy
            # This can be risky if the video is much shorter; it will freeze.
            # But if they are close, it can align them.
            # Alternatively, ensure `timeline` (final video duration) is used to trim/pad audio
            # or video is trimmed/padded to `target_duration`.
            # The most robust is usually to ensure video stream is at least as long as audio.
            # If video is shorter, it will freeze. If longer, audio will end and video continues (silent).
            # If audio is longer, video will freeze.
            # FFmpeg default behavior: output duration = duration of longest input.
            # We want output duration = target_duration.
            # We can explicitly set the output duration using -t for the output,
            # but it's better to get the stream lengths correct.
            # Adding `-shortest` would make the output duration that of the shortest input stream.
            # If padded_audio is target_duration and video timeline is target_duration, it's fine.
            # If video timeline is slightly less, -shortest would use video timeline.
            # Forcing output duration: add `t=target_duration` to the output options.
            # e.g. ffmpeg.output(..., t=target_duration, ...)
        )
        print(f"  Writing reel: {output_file_path} (target output duration: {target_duration:.2f}s)")
        # For debugging, you can print the command: print("FFMPEG command:", out_node.compile())
        ffmpeg.run(out_node, overwrite_output=True, quiet=False) # quiet=False for more ffmpeg output
        print(f"Completed: {output_file_path}")
        return output_file_path
    except ffmpeg.Error as e:
        print(f"Error writing {output_file_path}: {e}")
        if e.stderr:
            print(f"FFmpeg stderr: {e.stderr.decode('utf8')}")
        return None
    except Exception as e_gen:
        print(f"A general error occurred during output for {output_file_path}: {e_gen}")
        return None

def concatenate_sections(section_files, final_output):
    # This function remains the same
    list_path = os.path.join(os.path.dirname(final_output), 'sections_to_concat.txt')
    # Ensure absolute paths are used in the concat file list, especially if temp dirs are involved
    with open(list_path, 'w') as f:
        for filepath in section_files:
            # FFmpeg concat demuxer expects paths to be relative to the list file, or absolute.
            # Using absolute paths is safer.
            abs_filepath = os.path.abspath(filepath)
            # FFmpeg on Windows might have issues with backslashes in concat file list if not escaped.
            # For safety, replace backslashes with forward slashes.
            abs_filepath = abs_filepath.replace('\\', '/')
            f.write(f"file '{abs_filepath}'\n")


    print(f"Concatenating {len(section_files)} sections into {final_output}")
    try:
        (
            ffmpeg
            .input(list_path, format='concat', safe=0)
            .output(
                final_output,
                vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC, # Use same codecs
                audio_bitrate=AUDIO_BITRATE, # Use same audio bitrate
                crf=CRF, preset=PRESET_X264, # Use same video quality settings
                movflags='+faststart'
            )
            .run(overwrite_output=True, quiet=False) # quiet=False for more ffmpeg output
        )
        print(f"Final reel created: {final_output}")
    except ffmpeg.Error as e:
        print(f"Error concatenating final reel: {e}")
        if e.stderr:
            print(f"FFmpeg stderr: {e.stderr.decode('utf8')}")
    except Exception as e_gen:
         print(f"A general error occurred during final concatenation: {e_gen}")


def main():
    # This function remains the same
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    scripts = [d for d in os.listdir(BASE_VIDEOS_PATH)
               if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script in scripts:
        base = os.path.join(BASE_VIDEOS_PATH, script)
        audio_dir = os.path.join(base, 'audio')
        visuals_root = os.path.join(base, 'visuals') # Root for general visuals
        out_root = os.path.join(OUTPUT_DIR_BASE, script)
        os.makedirs(out_root, exist_ok=True)

        if not os.path.isdir(audio_dir):
            print(f"Audio directory not found for script {script}: {audio_dir}")
            continue
        # Visuals_root might not exist if all visuals are section-specific,
        # but the logic below handles it.

        print(f"\nProcessing Script folder: {script}")
        section_outputs = []
        
        # Ensure audio files are processed in a consistent order (e.g., alphabetically)
        # if the order of sections matters for the final concatenated reel.
        audio_files_sorted = sorted([f for f in os.listdir(audio_dir) 
                                     if f.lower().endswith(('.wav', '.mp3', '.m4a', '.aac'))])

        for audio_file in audio_files_sorted:
            vids_for_section = []
            section_name_from_audio = os.path.splitext(audio_file)[0]
            
            # Try to find section-specific visuals first
            section_visuals_folder = os.path.join(visuals_root, f'section_{section_name_from_audio}') # Matches user's naming
            
            potential_visual_sources = []
            if os.path.isdir(section_visuals_folder):
                potential_visual_sources.append(section_visuals_folder)
            
            # Add visuals from the general 'visuals_root' if it exists
            if os.path.isdir(visuals_root): # Check if general visuals_root exists
                potential_visual_sources.append(visuals_root)

            if not potential_visual_sources:
                print(f"No visual sources (section-specific or general) found for section {section_name_from_audio} in script {script}.")
                # Decide if to skip or proceed without visuals (current logic would fail in create_reel_for_audio if associated_video_files is empty)
                # For now, let's ensure create_reel_for_audio handles empty associated_video_files gracefully or we skip here.
                # The check `if not associated_video_files:` in `create_reel_for_audio` handles this.
                pass # Will pass empty vids_for_section

            for source_dir in potential_visual_sources:
                vids_for_section.extend([
                    os.path.join(source_dir, f)
                    for f in os.listdir(source_dir)
                    if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv', '.webm')) # Added .webm
                ])
            
            # Remove duplicates if a file could be in multiple specified paths (though unlikely with current structure)
            vids_for_section = sorted(list(set(vids_for_section)))


            if not vids_for_section:
                print(f"No video files found for section {section_name_from_audio} in script {script}. Searched in: {potential_visual_sources}")
                # Continue to next audio file if no visuals are found for this one.
                # create_reel_for_audio will also return None if associated_video_files is empty.

            audio_path = os.path.join(audio_dir, audio_file)
            # Ensure section name in output file is sanitized if it contains special characters
            safe_section_name = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in section_name_from_audio)
            out_path = os.path.join(out_root, f'reel_{safe_section_name}.mp4')
            
            result = create_reel_for_audio(audio_path, vids_for_section, out_path)
            if result:
                section_outputs.append(result)

        if section_outputs:
            final_out_safe_script_name = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in script)
            final_out = os.path.join(out_root, f'{final_out_safe_script_name}_final_reel.mp4')
            concatenate_sections(section_outputs, final_out)
        elif not audio_files_sorted:
             print(f"No audio files found to process in script {script}.")
        else:
            print(f"No sections were successfully created for script {script}. Final reel will not be generated.")


if __name__ == '__main__':
    # Ensure BASE_VIDEOS_PATH exists before starting
    if not os.path.isdir(BASE_VIDEOS_PATH):
        print(f"Error: Base videos path not found: {BASE_VIDEOS_PATH}")
        exit(1)
    main()


Processing Script folder: Why_You_Underestimate_RisksAnd_How_It_Costs_You

Processing ACTIONABLE_TIP: audio 15.89s + silence 1.20s = target 17.09s
  Segment from ACTIONABLE_TIP_6000421.mp4: ss=1.58, raw_t=5.00s
  Segment from ACTIONABLE_TIP_6781988.mp4: ss=2.93, raw_t=5.00s
  Segment from ACTIONABLE_TIP_7196309.mp4: ss=0.09, raw_t=5.00s
  Segment from ACTIONABLE_TIP_7062989.mp4: ss=9.06, raw_t=3.59s
  Collected 4 segments. Total effective video length: 17.09s (target 17.09s)
    Applying xfade: transition=fade, duration=0.50s, offset=4.50s
    Applying xfade: transition=fade, duration=0.50s, offset=9.00s
    Applying xfade: transition=fade, duration=0.50s, offset=13.50s
  Final calculated video duration after xfades should be: 17.09s
  Writing reel: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_ACTIONABLE_TIP.mp4 (target output duration: 17.09s)


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_ACTIONABLE_TIP.mp4

Processing CONCEPT: audio 15.25s + silence 1.30s = target 16.55s
  Segment from CONCEPT_8544030.mp4: ss=2.70, raw_t=5.00s
  Segment from CONCEPT_8544033.mp4: ss=5.15, raw_t=5.00s
  Segment from CONCEPT_8543925.mp4: ss=3.93, raw_t=5.00s
  Segment from CONCEPT_8544030.mp4: ss=3.09, raw_t=3.05s
  Collected 4 segments. Total effective video length: 16.55s (target 16.55s)
    Applying xfade: transition=fade, duration=0.50s, offset=4.50s
    Applying xfade: transition=fade, duration=0.50s, offset=9.00s
    Applying xfade: transition=fade, duration=0.50s, offset=13.50s
  Final calculated video duration after xfades should be: 16.55s
  Writing reel: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_CONCEPT.mp4 (target output duration: 16.55s)


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_CONCEPT.mp4

Processing CTA: audio 5.38s + silence 0.80s = target 6.18s
  Segment from CTA_9943144.mp4: ss=0.14, raw_t=5.00s
  Segment from CTA_15019286.mp4: ss=1.18, raw_t=1.68s
  Collected 2 segments. Total effective video length: 6.18s (target 6.18s)
    Applying xfade: transition=fade, duration=0.50s, offset=4.50s
  Final calculated video duration after xfades should be: 6.18s
  Writing reel: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_CTA.mp4 (target output duration: 6.18s)


...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/reel_REAL-WORLD_EXAMPLE.mp4
Concatenating 6 sections into output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/Why_You_Underestimate_RisksAnd_How_It_Costs_You_final_reel.mp4


Input #0, concat, from 'output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/sections_to_concat.txt':
  Duration: N/A, start: -0.042667, bitrate: 936 kb/s
  Stream #0:0(und): Video: h264 (High 4:4:4 Predictive) (avc1 / 0x31637661), yuv444p(tv, bt709, progressive), 720x1280, 836 kb/s, 30 fps, 30 tbr, 15360 tbn
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
      encoder         : Lavc60.31.102 libx264
  Stream #0:1(und): Audio: aac (LC) (mp4a / 0x6134706D), 24000 Hz, mono, fltp, 100 kb/s
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x5c67a1712f40] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5c67a1712f40] profile High 4:4:4 Predictive, level 3.1, 4:4:4, 8-bit
[libx264 @ 0x5c67a171

Final reel created: output_reels_fades_only_v2/Why_You_Underestimate_RisksAnd_How_It_Costs_You/Why_You_Underestimate_RisksAnd_How_It_Costs_You_final_reel.mp4


[mp4 @ 0x5c67a171e880] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x5c67a171b2c0] video:13065kB audio:1374kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.615174%
frame= 3374 fps= 69 q=-1.0 Lsize=   14527kB time=00:01:52.45 bitrate=1058.2kbits/s dup=2 drop=0 speed=2.31x    
[libx264 @ 0x5c67a1712f40] frame I:21    Avg QP:18.28  size: 28575
[libx264 @ 0x5c67a1712f40] frame P:999   Avg QP:20.88  size:  8989
[libx264 @ 0x5c67a1712f40] frame B:2354  Avg QP:24.18  size:  1613
[libx264 @ 0x5c67a1712f40] consecutive B-frames:  0.9% 16.7%  4.6% 77.8%
[libx264 @ 0x5c67a1712f40] mb I  I16..4: 21.0% 73.0%  5.9%
[libx264 @ 0x5c67a1712f40] mb P  I16..4:  4.0%  9.4%  0.5%  P16..4: 35.5%  7.7%  3.0%  0.0%  0.0%    skip:39.9%
[libx264 @ 0x5c67a1712f40] mb B  I16..4:  0.2%  0.4%  0.0%  B16..8: 26.2%  0.8%  0.1%  direct: 0.3%  skip:71.9%  L0:47.6% L1:49.2% BI: 3.1%
[libx264 @ 0x5c67a1712f40] 8x8 transform intra:67.9% inter:90.7%
[libx264 @

In [4]:
import ffmpeg
import random
import os

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/" # Adjust as needed
OUTPUT_DIR_BASE = "output_reels_fades_ordered_v3" # New output folder

MIN_SEGMENT_DURATION = 3.0
MAX_SEGMENT_DURATION = 8.0
TRANSITION_DURATION = 0.5 # Used for inter-segment fades and final CTA fade-out
REEL_WIDTH = 480            # Lower resolution
REEL_HEIGHT = 854
OUTPUT_FPS = 24  

VIDEO_CODEC = 'libx264'
PRESET = 'ultrafast'
CRF = 35
VIDEO_BITRATE = '800k'
AUDIO_CODEC = 'aac'
AUDIO_BITRATE = '96k'

CRF = 23
PRESET_X264 = 'ultrafast'
XFADE_TRANSITIONS = ['fade']

SECTION_SILENCE = {
    'HOOK': 1.0,
    'CONCEPT': 1.3,
    'REAL-WORLD_EXAMPLE': 1.5,
    'PSYCHOLOGICAL_INSIGHT': 2.0,
    'ACTIONABLE_TIP': 1.2,
    'CTA': 0.8
}

# Define the desired order for final concatenation
SECTION_ORDER = ['HOOK', 'CONCEPT', 'REAL-WORLD_EXAMPLE', 'PSYCHOLOGICAL_INSIGHT', 'ACTIONABLE_TIP', 'CTA']


def get_duration(filename):
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except Exception as e:
        print(f"Error probing duration for {filename}: {e}")
        return None


def apply_segment_effects(video_node):
    node = video_node.filter(
        'scale', w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
        h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)', eval='frame'
    )
    node = node.filter('crop', w=REEL_WIDTH, h=REEL_HEIGHT,
                       x=f'(iw-{REEL_WIDTH})/2', y=f'(ih-{REEL_HEIGHT})/2')
    node = node.filter('fps', fps=OUTPUT_FPS, round='near')
    node = node.filter('setpts', 'PTS-STARTPTS')
    return node


def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    section_name = os.path.splitext(os.path.basename(audio_file_path))[0]
    section_key = section_name.upper() # Key for SECTION_SILENCE and CTA check
    silence_duration = SECTION_SILENCE.get(section_key, 1.0)

    # 1. Measure audio
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None:
        print(f"Skipping {audio_file_path}, could not get audio duration.")
        return None
    if not associated_video_files:
        print(f"Skipping {section_name} ({audio_file_path}), no associated video files.")
        return None

    target_duration = audio_duration + silence_duration
    print(f"\nProcessing {section_name}: audio {audio_duration:.2f}s + silence {silence_duration:.2f}s = target {target_duration:.2f}s")

    # 2. Collect segments until filling target_duration
    segments = []
    effective_video_duration = 0.0
    files_to_use = associated_video_files.copy()
    random.shuffle(files_to_use)
    max_segment_picking_attempts = len(files_to_use) * 3 + 5
    current_attempt_idx = 0
    file_picker_idx = 0

    while effective_video_duration < target_duration and current_attempt_idx < max_segment_picking_attempts:
        current_attempt_idx += 1
        if not files_to_use: break

        video_path = files_to_use[file_picker_idx % len(files_to_use)]
        source_video_duration = get_duration(video_path) or 0.0
        needed_effective_contribution = target_duration - effective_video_duration
        if needed_effective_contribution <= 0.01: break

        segment_raw_duration = 0.0
        is_first_segment = not bool(segments)
        min_raw_len_this_segment = MIN_SEGMENT_DURATION
        max_raw_len_this_segment = MAX_SEGMENT_DURATION

        if is_first_segment:
            ideal_len = min(needed_effective_contribution, max_raw_len_this_segment)
            segment_raw_duration = max(ideal_len, min_raw_len_this_segment if ideal_len >= min_raw_len_this_segment else ideal_len)
            segment_raw_duration = min(segment_raw_duration, source_video_duration)
        else:
            min_raw_len_this_segment = max(MIN_SEGMENT_DURATION, TRANSITION_DURATION + 0.01)
            target_raw_len_for_segment = needed_effective_contribution + TRANSITION_DURATION
            ideal_len = min(target_raw_len_for_segment, max_raw_len_this_segment)
            segment_raw_duration = max(ideal_len, min_raw_len_this_segment if ideal_len >= min_raw_len_this_segment else ideal_len)
            segment_raw_duration = min(segment_raw_duration, source_video_duration)

        valid_segment = True
        if segment_raw_duration < 0.1: valid_segment = False
        if not is_first_segment and segment_raw_duration <= TRANSITION_DURATION: valid_segment = False
        
        if not valid_segment:
            print(f"  Skipping {os.path.basename(video_path)} for {section_name}: calculated raw_t ({segment_raw_duration:.2f}s) unsuitable. Source_t: {source_video_duration:.2f}s.")
            file_picker_idx += 1
            continue

        start_time = random.uniform(0, max(0, source_video_duration - segment_raw_duration))
        print(f"  Segment for {section_name} from {os.path.basename(video_path)}: ss={start_time:.2f}, raw_t={segment_raw_duration:.2f}s")
        inp = ffmpeg.input(video_path, ss=start_time, t=segment_raw_duration)
        vid_node = apply_segment_effects(inp.video)
        segments.append({'stream': vid_node, 'raw_duration': segment_raw_duration})

        if is_first_segment:
            effective_video_duration += segment_raw_duration
        else:
            effective_video_duration += segment_raw_duration - TRANSITION_DURATION
        file_picker_idx += 1

    if not segments:
        print(f"Error: No video segments collected for {section_name} ({audio_file_path}). Target: {target_duration:.2f}s, Effective Found: {effective_video_duration:.2f}s")
        return None
    print(f"  Collected {len(segments)} segments for {section_name}. Effective video length: {effective_video_duration:.2f}s (target {target_duration:.2f}s)")

    # 3. Concatenate video segments with crossfades
    merged_video_stream = segments[0]['stream']
    timeline = segments[0]['raw_duration'] 
    for i in range(1, len(segments)):
        next_segment_data = segments[i]
        xfade_offset = max(0, timeline - TRANSITION_DURATION) 
        print(f"    Applying xfade to {section_name}: transition={XFADE_TRANSITIONS[0]}, duration={TRANSITION_DURATION:.2f}s, offset={xfade_offset:.2f}s")
        merged_video_stream = ffmpeg.filter(
            [merged_video_stream, next_segment_data['stream']], 'xfade',
            transition=XFADE_TRANSITIONS[0], duration=TRANSITION_DURATION, offset=xfade_offset
        ).filter('setpts', 'PTS-STARTPTS')
        timeline += next_segment_data['raw_duration'] - TRANSITION_DURATION
    print(f"  Video duration for {section_name} after xfades (before CTA fade): {timeline:.2f}s")

    # Apply fade-to-black for CTA section video stream
    if section_key == 'CTA':
        print(f"  Applying fade-to-black at the end of CTA section video.")
        # timeline is the actual duration of the video content for the CTA section
        fade_start_time = max(0, timeline - TRANSITION_DURATION)
        actual_fade_duration = timeline - fade_start_time
        
        if actual_fade_duration > 0.01: # Only apply if meaningful duration
            merged_video_stream = merged_video_stream.filter(
                'fade', type='out',
                start_time=fade_start_time,
                duration=actual_fade_duration,
                color='black' 
            )
            print(f"    CTA fade-out applied: start_time={fade_start_time:.2f}s, duration={actual_fade_duration:.2f}s. Video will end black.")
        else:
            print(f"    CTA video content ({timeline:.2f}s) too short for effective fade-out. Skipping fade for {section_name}.")

    # 4. Build padded audio
    tts_input = ffmpeg.input(audio_file_path)
    silence_input = ffmpeg.input(
        'anullsrc=channel_layout=stereo:sample_rate=44100', format='lavfi', t=silence_duration
    )
    padded_audio = ffmpeg.filter([tts_input.audio, silence_input.audio], 'concat', n=2, v=0, a=1)

    # 5. Merge video & audio for the section
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    try:
        out_node = ffmpeg.output(
            merged_video_stream, padded_audio, output_file_path,
            vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC, audio_bitrate=AUDIO_BITRATE,
            crf=CRF, preset=PRESET_X264, movflags='+faststart',
        )
        print(f"  Writing section reel: {output_file_path} (target section duration: {target_duration:.2f}s)")
        ffmpeg.run(out_node, overwrite_output=True, quiet=False)
        print(f"Completed section: {output_file_path}")
        return output_file_path
    except ffmpeg.Error as e:
        print(f"Error writing section {output_file_path}: {e}")
        if e.stderr: print(f"FFmpeg stderr: {e.stderr.decode('utf8')}")
        return None
    except Exception as e_gen:
        print(f"General error during output for section {output_file_path}: {e_gen}")
        return None


def concatenate_sections(section_files, final_output):
    list_path = os.path.join(os.path.dirname(final_output), 'sections_to_concat.txt')
    with open(list_path, 'w') as f:
        for filepath in section_files:
            abs_filepath = os.path.abspath(filepath).replace('\\', '/')
            f.write(f"file '{abs_filepath}'\n")

    print(f"Concatenating {len(section_files)} sections into {final_output}")
    try:
        (
            ffmpeg
            .input(list_path, format='concat', safe=0)
            .output(
                final_output, vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
                audio_bitrate=AUDIO_BITRATE, crf=CRF, preset=PRESET_X264,
                movflags='+faststart'
            )
            .run(overwrite_output=True, quiet=False)
        )
        print(f"Final reel created: {final_output}")
    except ffmpeg.Error as e:
        print(f"Error concatenating final reel: {e}")
        if e.stderr: print(f"FFmpeg stderr: {e.stderr.decode('utf8')}")
    except Exception as e_gen:
         print(f"General error during final concatenation: {e_gen}")


def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    if not os.path.isdir(BASE_VIDEOS_PATH):
        print(f"Error: Base videos path not found: {BASE_VIDEOS_PATH}")
        return

    scripts = [d for d in os.listdir(BASE_VIDEOS_PATH)
               if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script in scripts:
        base_script_path = os.path.join(BASE_VIDEOS_PATH, script)
        audio_dir = os.path.join(base_script_path, 'audio')
        visuals_root = os.path.join(base_script_path, 'visuals')
        output_script_root = os.path.join(OUTPUT_DIR_BASE, script)
        os.makedirs(output_script_root, exist_ok=True)

        if not os.path.isdir(audio_dir):
            print(f"Audio directory not found for script {script}: {audio_dir}. Skipping script.")
            continue
        print(f"\nProcessing Script folder: {script}")

        # Store paths to generated section reels, mapped by section key
        section_output_map = {} 
        
        # Process audio files to generate sections
        available_audio_files = sorted([f for f in os.listdir(audio_dir) 
                                     if f.lower().endswith(('.wav', '.mp3', '.m4a', '.aac'))])
        if not available_audio_files:
            print(f"No audio files found in {audio_dir} for script {script}. Skipping script.")
            continue

        for audio_file in available_audio_files:
            section_name_from_audio = os.path.splitext(audio_file)[0]
            section_key_from_audio = section_name_from_audio.upper() # For matching SECTION_ORDER

            vids_for_section = []
            section_visuals_folder = os.path.join(visuals_root, f'section_{section_name_from_audio}')
            
            potential_visual_sources = []
            if os.path.isdir(section_visuals_folder):
                potential_visual_sources.append(section_visuals_folder)
            if os.path.isdir(visuals_root):
                potential_visual_sources.append(visuals_root)

            for source_dir in potential_visual_sources:
                vids_for_section.extend([
                    os.path.join(source_dir, f)
                    for f in os.listdir(source_dir)
                    if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv', '.webm'))
                ])
            vids_for_section = sorted(list(set(vids_for_section)))

            audio_path = os.path.join(audio_dir, audio_file)
            safe_section_name = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in section_name_from_audio)
            output_section_reel_path = os.path.join(output_script_root, f'reel_{safe_section_name}.mp4')
            
            # The create_reel_for_audio function internally derives section_key for CTA fade
            result_path = create_reel_for_audio(audio_path, vids_for_section, output_section_reel_path)
            if result_path:
                section_output_map[section_key_from_audio] = result_path
        
        # After processing all audio files for the current script, prepare ordered list for concatenation
        ordered_section_files_for_concat = []
        print(f"\nPreparing final reel for script '{script}' based on order: {SECTION_ORDER}")
        for section_key_in_order in SECTION_ORDER:
            if section_key_in_order in section_output_map:
                ordered_section_files_for_concat.append(section_output_map[section_key_in_order])
                print(f"  + Added section '{section_key_in_order}'")
            else:
                print(f"  - Warning: Reel for section '{section_key_in_order}' is missing for script '{script}'. It will be excluded.")

        if ordered_section_files_for_concat:
            if len(ordered_section_files_for_concat) < len(SECTION_ORDER):
                 print(f"Warning: Concatenating {len(ordered_section_files_for_concat)} available sections for '{script}' (some were missing).")
            
            final_out_safe_script_name = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in script)
            final_reel_output_path = os.path.join(output_script_root, f'{final_out_safe_script_name}_final_ordered_reel.mp4')
            concatenate_sections(ordered_section_files_for_concat, final_reel_output_path)
        elif not available_audio_files:
             # This case is handled earlier by skipping the script.
             pass
        else: # Some audio files were processed, but none matched the required sections or failed.
            print(f"No sections were successfully processed and matched the defined order for script '{script}'. Final ordered reel will not be generated.")


if __name__ == '__main__':
    main()


Processing Script folder: Why_You_Think_Someone_Elses_Success_Means_Your_Loss

Processing ACTIONABLE_TIP: audio 14.49s + silence 1.20s = target 15.69s
  Segment for ACTIONABLE_TIP from ACTIONABLE_TIP_8724454.mp4: ss=8.92, raw_t=8.00s
  Segment for ACTIONABLE_TIP from ACTIONABLE_TIP_7581329.mp4: ss=0.14, raw_t=8.00s
  Segment for ACTIONABLE_TIP from ACTIONABLE_TIP_9034459.mp4: ss=8.24, raw_t=0.69s
  Collected 3 segments for ACTIONABLE_TIP. Effective video length: 15.69s (target 15.69s)
    Applying xfade to ACTIONABLE_TIP: transition=fade, duration=0.50s, offset=7.50s
    Applying xfade to ACTIONABLE_TIP: transition=fade, duration=0.50s, offset=15.00s
  Video duration for ACTIONABLE_TIP after xfades (before CTA fade): 15.69s
  Writing section reel: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/reel_ACTIONABLE_TIP.mp4 (target section duration: 15.69s)


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed section: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/reel_ACTIONABLE_TIP.mp4

Processing CONCEPT: audio 12.33s + silence 1.30s = target 13.63s
  Segment for CONCEPT from CONCEPT_5989764.mp4: ss=3.88, raw_t=8.00s
  Segment for CONCEPT from CONCEPT_7581329.mp4: ss=1.69, raw_t=6.13s
  Collected 2 segments for CONCEPT. Effective video length: 13.63s (target 13.63s)
    Applying xfade to CONCEPT: transition=fade, duration=0.50s, offset=7.50s
  Video duration for CONCEPT after xfades (before CTA fade): 13.63s
  Writing section reel: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/reel_CONCEPT.mp4 (target section duration: 13.63s)


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed section: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/reel_CONCEPT.mp4

Processing CTA: audio 5.10s + silence 0.80s = target 5.90s
  Segment for CTA from CTA_7578123.mp4: ss=0.36, raw_t=5.90s
  Collected 1 segments for CTA. Effective video length: 5.90s (target 5.90s)
  Video duration for CTA after xfades (before CTA fade): 5.90s
  Applying fade-to-black at the end of CTA section video.
    CTA fade-out applied: start_time=5.40s, duration=0.50s. Video will end black.
  Writing section reel: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/reel_CTA.mp4 (target section duration: 5.90s)


...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed section: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/reel_REAL-WORLD_EXAMPLE.mp4

Preparing final reel for script 'Why_You_Think_Someone_Elses_Success_Means_Your_Loss' based on order: ['HOOK', 'CONCEPT', 'REAL-WORLD_EXAMPLE', 'PSYCHOLOGICAL_INSIGHT', 'ACTIONABLE_TIP', 'CTA']
  + Added section 'HOOK'
  + Added section 'CONCEPT'
  + Added section 'REAL-WORLD_EXAMPLE'
  + Added section 'PSYCHOLOGICAL_INSIGHT'
  + Added section 'ACTIONABLE_TIP'
  + Added section 'CTA'
Concatenating 6 sections into output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/Why_You_Think_Someone_Elses_Success_Means_Your_Loss_final_ordered_reel.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Final reel created: output_reels_fades_ordered_v3/Why_You_Think_Someone_Elses_Success_Means_Your_Loss/Why_You_Think_Someone_Elses_Success_Means_Your_Loss_final_ordered_reel.mp4


[mp4 @ 0x5869c4f09140] Starting second pass: moving the moov atom to the beginning of the file
[out#0/mp4 @ 0x5869c4f52500] video:28682kB audio:1105kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.125853%
frame= 2263 fps=333 q=-1.0 Lsize=   29824kB time=00:01:34.25 bitrate=2592.2kbits/s dup=2 drop=0 speed=13.9x    
[libx264 @ 0x5869c4fa1800] frame I:10    Avg QP:18.30  size: 67772
[libx264 @ 0x5869c4fa1800] frame P:2253  Avg QP:20.52  size: 12735
[libx264 @ 0x5869c4fa1800] mb I  I16..4: 100.0%  0.0%  0.0%
[libx264 @ 0x5869c4fa1800] mb P  I16..4:  5.2%  0.0%  0.0%  P16..4: 48.0%  0.0%  0.0%  0.0%  0.0%    skip:46.8%
[libx264 @ 0x5869c4fa1800] coded y,u,v intra: 35.3% 2.5% 2.3% inter: 27.0% 2.1% 1.9%
[libx264 @ 0x5869c4fa1800] i16 v,h,dc,p: 41% 22% 19% 18%
[libx264 @ 0x5869c4fa1800] kb/s:2491.82
[aac @ 0x5869c57880c0] Qavg: 18893.172
